# 🌟 CAMADA GOLD - Star Schema para Análise de Surf
## Modelagem dimensional otimizada para BI e análises de negócio

### 🎯 **Objetivo da Camada Gold:**
- Criar modelo Star Schema para análises de surf
- Implementar tabela fato `fact_surf_conditions`
- Desenvolver dimensões especializadas (`dim_date`, `dim_weather`, `dim_swell`)
- Calcular métricas de negócio (Surf Quality Index, Session Rating)
- Dados prontos para BI tools (Tableau, Power BI, etc.)

### 📊 **Arquitetura do Star Schema:**
```
        DIM_DATE
            |
            |
DIM_WEATHER → FACT_SURF_CONDITIONS ← DIM_SWELL
            |
        DIM_LOCATION
```

In [3]:
# ==============================================================================
# IMPORTS E CONFIGURAÇÃO INICIAL
# ==============================================================================

import pandas as pd
import numpy as np
import duckdb
from deltalake import DeltaTable
from deltalake.writer import write_deltalake
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Define os caminhos para as camadas
SILVER_DIR = "data/silver"
GOLD_DIR = "data/gold"

# Cria o diretório da camada Gold se não existir
os.makedirs(GOLD_DIR, exist_ok=True)

# Inicia conexão DuckDB
con = duckdb.connect()

print("🌟 INICIANDO CRIAÇÃO DA CAMADA GOLD")
print("=" * 50)
print(f"Silver Dir: {SILVER_DIR}")
print(f"Gold Dir: {GOLD_DIR}")
print("=" * 50)

🌟 INICIANDO CRIAÇÃO DA CAMADA GOLD
Silver Dir: data/silver
Gold Dir: data/gold


In [7]:
# ==============================================================================
# CARREGAMENTO DAS TABELAS SILVER
# ==============================================================================

print("📊 CARREGANDO TABELAS SILVER...")

# Carrega todas as tabelas Delta da camada Silver
tables_silver = {}
silver_tables_info = []

for table_name in ['wind', 'meteomatics', 'rating', 'surf', 'swells', 'tides', 'sunlight']:
    table_path = os.path.join(SILVER_DIR, table_name)
    if os.path.exists(table_path):
        dt = DeltaTable(table_path)
        df = dt.to_pandas()
        tables_silver[table_name] = df
        
        silver_tables_info.append({
            'Tabela': table_name,
            'Registros': len(df),
            'Colunas': len(df.columns),
            'Período': f"{df.iloc[:, 0].min()} até {df.iloc[:, 0].max()}" if len(df) > 0 else "N/A"
        })
        
        # Registra no DuckDB para uso em SQL
        con.register(f'silver_{table_name}', df)
        print(f"✅ {table_name}: {len(df):,} registros")

print("\n📋 RESUMO DAS TABELAS CARREGADAS:")
for info in silver_tables_info:
    print(f"• {info['Tabela']}: {info['Registros']:,} registros, {info['Colunas']} colunas")

# Verifica tabelas disponíveis no DuckDB
print("\n🔍 TABELAS REGISTRADAS NO DUCKDB:")
available_tables = con.execute("SHOW TABLES;").df()
display(available_tables)

📊 CARREGANDO TABELAS SILVER...
✅ wind: 26,496 registros
✅ meteomatics: 26,305 registros
✅ rating: 26,304 registros
✅ surf: 26,304 registros
✅ swells: 26,304 registros
✅ tides: 32,885 registros
✅ sunlight: 1,096 registros

📋 RESUMO DAS TABELAS CARREGADAS:
• wind: 26,496 registros, 7 colunas
• meteomatics: 26,305 registros, 9 colunas
• rating: 26,304 registros, 4 colunas
• surf: 26,304 registros, 11 colunas
• swells: 26,304 registros, 35 colunas
• tides: 32,885 registros, 8 colunas
• sunlight: 1,096 registros, 17 colunas

🔍 TABELAS REGISTRADAS NO DUCKDB:
✅ meteomatics: 26,305 registros
✅ rating: 26,304 registros
✅ surf: 26,304 registros
✅ swells: 26,304 registros
✅ tides: 32,885 registros
✅ sunlight: 1,096 registros

📋 RESUMO DAS TABELAS CARREGADAS:
• wind: 26,496 registros, 7 colunas
• meteomatics: 26,305 registros, 9 colunas
• rating: 26,304 registros, 4 colunas
• surf: 26,304 registros, 11 colunas
• swells: 26,304 registros, 35 colunas
• tides: 32,885 registros, 8 colunas
• sunlight: 

,name
0,silver_meteomatics
1,silver_rating
2,silver_sunlight
3,silver_surf
4,silver_swells
5,silver_tides
6,silver_wind


# 🗓️ ETAPA 1: DIM_DATE - Dimensão Temporal
## Criando calendário completo com informações sazonais e de surf

In [5]:
# ==============================================================================
# CRIAÇÃO DA DIM_DATE
# ==============================================================================

# Query para criar dimensão temporal completa
query_dim_date = """
WITH date_range AS (
    -- Gera range de datas baseado nos dados disponíveis
    SELECT DISTINCT 
        CAST(datetime_utc AS DATE) as full_date
    FROM silver_surf  -- Usando surf como referência (tem boa cobertura)
    WHERE datetime_utc IS NOT NULL
),
date_attributes AS (
    SELECT 
        full_date,
        
        -- Componentes básicos de data
        EXTRACT(YEAR FROM full_date) as year,
        EXTRACT(MONTH FROM full_date) as month,
        EXTRACT(DAY FROM full_date) as day,
        EXTRACT(QUARTER FROM full_date) as quarter,
        EXTRACT(WEEK FROM full_date) as week_of_year,
        EXTRACT(DAYOFWEEK FROM full_date) as day_of_week,
        DAYNAME(full_date) as day_name,
        MONTHNAME(full_date) as month_name,
        
        -- Flags úteis
        CASE WHEN EXTRACT(DAYOFWEEK FROM full_date) IN (1, 7) THEN true ELSE false END as is_weekend,
        
        -- Estações do ano (hemisfério sul)
        CASE 
            WHEN EXTRACT(MONTH FROM full_date) IN (12, 1, 2) THEN 'Verão'
            WHEN EXTRACT(MONTH FROM full_date) IN (3, 4, 5) THEN 'Outono'
            WHEN EXTRACT(MONTH FROM full_date) IN (6, 7, 8) THEN 'Inverno'
            ELSE 'Primavera'
        END as season,
        
        -- Classificação para surf (baseada na estação)
        CASE 
            WHEN EXTRACT(MONTH FROM full_date) IN (12, 1, 2, 3) THEN 'Alta Temporada'
            WHEN EXTRACT(MONTH FROM full_date) IN (4, 5, 9, 10, 11) THEN 'Média Temporada'
            ELSE 'Baixa Temporada'
        END as surf_season,
        
        -- Período do ano
        CASE 
            WHEN EXTRACT(MONTH FROM full_date) BETWEEN 1 AND 6 THEN 'Primeiro Semestre'
            ELSE 'Segundo Semestre'
        END as half_year,
        
        -- Classificação de mês para análises
        CASE 
            WHEN EXTRACT(MONTH FROM full_date) IN (12, 1, 2) THEN 'Verão - Ondas Pequenas'
            WHEN EXTRACT(MONTH FROM full_date) IN (3, 4, 5) THEN 'Outono - Swells Consistentes'
            WHEN EXTRACT(MONTH FROM full_date) IN (6, 7, 8) THEN 'Inverno - Ondas Grandes'
            ELSE 'Primavera - Condições Variáveis'
        END as surf_period_description
        
    FROM date_range
)
SELECT 
    -- Chave primária
    ROW_NUMBER() OVER (ORDER BY full_date) as date_key,
    
    -- Todos os atributos
    *,
    
    -- Formatações adicionais
    CONCAT(year, '-', LPAD(CAST(month AS VARCHAR), 2, '0')) as year_month,
    CONCAT(year, '-Q', quarter) as year_quarter
    
FROM date_attributes
ORDER BY full_date
"""

# Executa a query e cria a dimensão
dim_date = con.execute(query_dim_date).fetch_df()

print("🗓️ DIM_DATE CRIADA COM SUCESSO!")
print(f"Registros: {len(dim_date):,}")
print(f"Período: {dim_date['full_date'].min()} até {dim_date['full_date'].max()}")
print("\nPrimeiras linhas:")
display(dim_date.head())

print("\nDistribuição por estação:")
print(dim_date['season'].value_counts())

print("\nDistribuição por temporada de surf:")
print(dim_date['surf_season'].value_counts())

🗓️ DIM_DATE CRIADA COM SUCESSO!
Registros: 1,097
Período: 2020-01-01 00:00:00 até 2023-01-01 00:00:00

Primeiras linhas:


,date_key,full_date,year,month,day,quarter,week_of_year,day_of_week,day_name,month_name,is_weekend,season,surf_season,half_year,surf_period_description,year_month,year_quarter
0,1,2020-01-01,2020,1,1,1,1,3,Wednesday,January,False,Verão,Alta Temporada,Primeiro Semestre,Verão - Ondas Pequenas,2020-01,2020-Q1
1,2,2020-01-02,2020,1,2,1,1,4,Thursday,January,False,Verão,Alta Temporada,Primeiro Semestre,Verão - Ondas Pequenas,2020-01,2020-Q1
2,3,2020-01-03,2020,1,3,1,1,5,Friday,January,False,Verão,Alta Temporada,Primeiro Semestre,Verão - Ondas Pequenas,2020-01,2020-Q1
3,4,2020-01-04,2020,1,4,1,1,6,Saturday,January,False,Verão,Alta Temporada,Primeiro Semestre,Verão - Ondas Pequenas,2020-01,2020-Q1
4,5,2020-01-05,2020,1,5,1,1,0,Sunday,January,False,Verão,Alta Temporada,Primeiro Semestre,Verão - Ondas Pequenas,2020-01,2020-Q1



Distribuição por estação:
season
Outono       276
Inverno      276
Primavera    273
Verão        272
Name: count, dtype: int64

Distribuição por temporada de surf:
surf_season
Média Temporada    456
Alta Temporada     365
Baixa Temporada    276
Name: count, dtype: int64


# 🌊 ETAPA 2: DIM_SWELL - Dimensão de Ondulações
## Classificações e categorias de swells para análise

In [9]:
# ==============================================================================
# CRIAÇÃO DA DIM_SWELL
# ==============================================================================

# Query para criar dimensão de swells
query_dim_swell = """
WITH swell_combinations AS (
    SELECT DISTINCT
        -- Características do swell dominante
        primary_swell_category,
        primary_swell_direction,
        swell_1_height_m,
        swell_1_period_s,
        swell_1_direction_deg,
        
        -- Categorização do período
        CASE 
            WHEN swell_1_period_s < 8 THEN 'Período Curto'
            WHEN swell_1_period_s < 12 THEN 'Período Médio'  
            WHEN swell_1_period_s < 16 THEN 'Período Longo'
            ELSE 'Período Muito Longo'
        END as period_category,
        
        -- Classificação de qualidade baseada em altura e período
        CASE 
            WHEN swell_1_height_m >= 1.5 AND swell_1_period_s >= 10 THEN 'Swell Épico'
            WHEN swell_1_height_m >= 1.0 AND swell_1_period_s >= 8 THEN 'Swell Bom'
            WHEN swell_1_height_m >= 0.5 AND swell_1_period_s >= 6 THEN 'Swell Regular'
            ELSE 'Swell Fraco'
        END as swell_quality,
        
        -- Potencial para surf
        CASE 
            WHEN swell_1_height_m >= 1.2 AND swell_1_period_s >= 9 
                 AND primary_swell_direction IN ('Sul', 'Sudeste') THEN 'Alto Potencial'
            WHEN swell_1_height_m >= 0.8 AND swell_1_period_s >= 7 THEN 'Médio Potencial'
            WHEN swell_1_height_m >= 0.4 THEN 'Baixo Potencial'
            ELSE 'Sem Potencial'
        END as surf_potential,
        
        -- Score numérico de 0-100
        LEAST(100, 
            (swell_1_height_m * 20) + 
            (swell_1_period_s * 3) + 
            (CASE WHEN primary_swell_direction IN ('Sul', 'Sudeste') THEN 20 ELSE 0 END)
        ) as swell_score
        
    FROM silver_swells
    WHERE swell_1_height_m IS NOT NULL 
      AND swell_1_period_s IS NOT NULL
),
unique_swells AS (
    SELECT DISTINCT
        primary_swell_category,
        primary_swell_direction,
        period_category,
        swell_quality,
        surf_potential,
        ROUND(AVG(swell_score), 1) as avg_swell_score
    FROM swell_combinations
    GROUP BY 1,2,3,4,5
)
SELECT 
    ROW_NUMBER() OVER (ORDER BY avg_swell_score DESC) as swell_key,
    primary_swell_category as height_category,
    primary_swell_direction as direction_category,
    period_category,
    swell_quality,
    surf_potential,
    avg_swell_score,
    
    -- Descrição combinada
    CONCAT(
        primary_swell_category, ' - ', 
        primary_swell_direction, ' - ', 
        period_category
    ) as swell_description,
    
    -- Classificação binária simplificada
    CASE 
        WHEN avg_swell_score >= 60 THEN 'Bom para Surf'
        WHEN avg_swell_score >= 30 THEN 'Regular para Surf'
        ELSE 'Ruim para Surf'
    END as simple_rating

FROM unique_swells
ORDER BY avg_swell_score DESC
"""

# Executa a query e cria a dimensão
dim_swell = con.execute(query_dim_swell).fetch_df()

print("🌊 DIM_SWELL CRIADA COM SUCESSO!")
print(f"Registros: {len(dim_swell):,}")
print("\nPrimeiras linhas:")
display(dim_swell.head(10))

print("\nDistribuição por qualidade de swell:")
print(dim_swell['swell_quality'].value_counts())

print("\nDistribuição por potencial de surf:")
print(dim_swell['surf_potential'].value_counts())

print("\nTop 5 melhores combinações de swell:")
display(dim_swell.head())

🌊 DIM_SWELL CRIADA COM SUCESSO!
Registros: 83

Primeiras linhas:


,swell_key,height_category,direction_category,period_category,swell_quality,surf_potential,avg_swell_score,swell_description,simple_rating
0,1,Gigante,Sul,Período Médio,Swell Bom,Alto Potencial,100.0,Gigante - Sul - Período Médio,Bom para Surf
1,2,Gigante,Sul,Período Longo,Swell Épico,Alto Potencial,100.0,Gigante - Sul - Período Longo,Bom para Surf
2,3,Gigante,Sul,Período Médio,Swell Épico,Alto Potencial,100.0,Gigante - Sul - Período Médio,Bom para Surf
3,4,Gigante,Sul,Período Médio,Swell Bom,Médio Potencial,100.0,Gigante - Sul - Período Médio,Bom para Surf
4,5,Grande,Sul,Período Muito Longo,Swell Épico,Alto Potencial,100.0,Grande - Sul - Período Muito Longo,Bom para Surf
5,6,Muito Grande,Sul,Período Muito Longo,Swell Épico,Alto Potencial,100.0,Muito Grande - Sul - Período Muito Longo,Bom para Surf
6,7,Muito Grande,Sul,Período Longo,Swell Épico,Alto Potencial,99.7,Muito Grande - Sul - Período Longo,Bom para Surf
7,8,Gigante,Leste,Período Médio,Swell Épico,Médio Potencial,97.2,Gigante - Leste - Período Médio,Bom para Surf
8,9,Gigante,Leste,Período Longo,Swell Épico,Médio Potencial,96.6,Gigante - Leste - Período Longo,Bom para Surf
9,10,Muito Grande,Sul,Período Médio,Swell Épico,Alto Potencial,96.5,Muito Grande - Sul - Período Médio,Bom para Surf



Distribuição por qualidade de swell:
swell_quality
Swell Regular    32
Swell Bom        20
Swell Épico      17
Swell Fraco      14
Name: count, dtype: int64

Distribuição por potencial de surf:
surf_potential
Médio Potencial    42
Baixo Potencial    28
Alto Potencial     13
Name: count, dtype: int64

Top 5 melhores combinações de swell:


,swell_key,height_category,direction_category,period_category,swell_quality,surf_potential,avg_swell_score,swell_description,simple_rating
0,1,Gigante,Sul,Período Médio,Swell Bom,Alto Potencial,100.0,Gigante - Sul - Período Médio,Bom para Surf
1,2,Gigante,Sul,Período Longo,Swell Épico,Alto Potencial,100.0,Gigante - Sul - Período Longo,Bom para Surf
2,3,Gigante,Sul,Período Médio,Swell Épico,Alto Potencial,100.0,Gigante - Sul - Período Médio,Bom para Surf
3,4,Gigante,Sul,Período Médio,Swell Bom,Médio Potencial,100.0,Gigante - Sul - Período Médio,Bom para Surf
4,5,Grande,Sul,Período Muito Longo,Swell Épico,Alto Potencial,100.0,Grande - Sul - Período Muito Longo,Bom para Surf


In [8]:
# ==============================================================================
# ANÁLISE DETALHADA DAS CATEGORIAS DE SWELLS
# ==============================================================================

print("🔍 INVESTIGANDO CATEGORIAS DE SWELLS...")

# Verifica as categorias únicas na tabela silver_swells
analysis_query = """
SELECT 
    primary_swell_category,
    primary_swell_direction,
    CASE 
        WHEN swell_1_period_s < 8 THEN 'Período Curto'
        WHEN swell_1_period_s < 12 THEN 'Período Médio'  
        WHEN swell_1_period_s < 16 THEN 'Período Longo'
        ELSE 'Período Muito Longo'
    END as period_category,
    COUNT(*) as freq
FROM silver_swells
WHERE swell_1_height_m IS NOT NULL 
  AND swell_1_period_s IS NOT NULL
GROUP BY 1,2,3
ORDER BY freq DESC
"""

categories_analysis = con.execute(analysis_query).fetch_df()

print(f"Total de combinações possíveis: {len(categories_analysis)}")
print("\nTop 10 combinações mais frequentes:")
display(categories_analysis.head(10))

print("\nDistribuição por categoria de altura:")
height_dist = con.execute("SELECT primary_swell_category, COUNT(*) as freq FROM silver_swells GROUP BY 1 ORDER BY freq DESC").fetch_df()
display(height_dist)

print("\nDistribuição por direção:")
direction_dist = con.execute("SELECT primary_swell_direction, COUNT(*) as freq FROM silver_swells GROUP BY 1 ORDER BY freq DESC").fetch_df()
display(direction_dist)

print("\nRange de alturas e períodos:")
range_query = """
SELECT 
    MIN(swell_1_height_m) as min_height,
    MAX(swell_1_height_m) as max_height,
    AVG(swell_1_height_m) as avg_height,
    MIN(swell_1_period_s) as min_period,
    MAX(swell_1_period_s) as max_period,
    AVG(swell_1_period_s) as avg_period
FROM silver_swells
WHERE swell_1_height_m IS NOT NULL AND swell_1_period_s IS NOT NULL
"""
ranges = con.execute(range_query).fetch_df()
display(ranges)

🔍 INVESTIGANDO CATEGORIAS DE SWELLS...
Total de combinações possíveis: 50

Top 10 combinações mais frequentes:


,primary_swell_category,primary_swell_direction,period_category,freq
0,Médio,Leste,Período Médio,4167
1,Médio,Sul,Período Médio,2982
2,Pequeno,Leste,Período Médio,2142
3,Médio,Leste,Período Curto,1956
4,Grande,Leste,Período Médio,1920
5,Pequeno,Leste,Período Curto,1824
6,Grande,Sul,Período Médio,1614
7,Pequeno,Sul,Período Médio,1506
8,Muito Grande,Sul,Período Médio,936
9,Muito Grande,Leste,Período Médio,900



Distribuição por categoria de altura:


,primary_swell_category,freq
0,Médio,10956
1,Pequeno,6573
2,Grande,5220
3,Muito Grande,3159
4,Gigante,342
5,Muito Pequeno,54



Distribuição por direção:


,primary_swell_direction,freq
0,Leste,13872
1,Sul,11454
2,Norte,960
3,Oeste,18



Range de alturas e períodos:


,min_height,max_height,avg_height,min_period,max_period,avg_period
0,0.4156,4.84063,1.386435,3,16,8.746807


# 🌤️ ETAPA 3: DIM_WEATHER - Dimensão Climática  
## Combinações de condições meteorológicas para surf

In [10]:
# ==============================================================================
# CRIAÇÃO DA DIM_WEATHER
# ==============================================================================

# Query para criar dimensão de condições climáticas
query_dim_weather = """
WITH weather_combinations AS (
    SELECT DISTINCT
        -- Vento (da tabela wind)
        CASE 
            WHEN w.wind_speed_kph <= 10 THEN 'Vento Fraco'
            WHEN w.wind_speed_kph <= 20 THEN 'Vento Moderado'
            WHEN w.wind_speed_kph <= 35 THEN 'Vento Forte'
            ELSE 'Vento Muito Forte'
        END as wind_category,
        
        CASE 
            WHEN w.wind_direction_deg BETWEEN 0 AND 45 OR w.wind_direction_deg BETWEEN 315 AND 360 THEN 'Norte'
            WHEN w.wind_direction_deg BETWEEN 46 AND 135 THEN 'Leste'
            WHEN w.wind_direction_deg BETWEEN 136 AND 225 THEN 'Sul'
            ELSE 'Oeste'
        END as wind_direction_category,
        
        -- Temperatura (da tabela meteomatics)
        CASE 
            WHEN m.temperature_2m_celsius < 15 THEN 'Frio'
            WHEN m.temperature_2m_celsius < 20 THEN 'Fresco'
            WHEN m.temperature_2m_celsius < 25 THEN 'Agradável'
            WHEN m.temperature_2m_celsius < 30 THEN 'Quente'
            ELSE 'Muito Quente'
        END as temperature_category,
        
        -- Pressão atmosférica
        CASE 
            WHEN m.pressure_msl_hpa < 1010 THEN 'Pressão Baixa'
            WHEN m.pressure_msl_hpa < 1020 THEN 'Pressão Normal'
            ELSE 'Pressão Alta'
        END as pressure_category,
        
        -- Precipitação
        CASE 
            WHEN m.precipitation_1h_mm = 0 THEN 'Sem Chuva'
            WHEN m.precipitation_1h_mm < 2 THEN 'Chuva Fraca'
            WHEN m.precipitation_1h_mm < 10 THEN 'Chuva Moderada'
            ELSE 'Chuva Forte'
        END as precipitation_category,
        
        -- Score de condições para surf (0-100)
        LEAST(100, 
            -- Vento: offshore é melhor
            (CASE 
                WHEN w.wind_direction_deg BETWEEN 225 AND 315 AND w.wind_speed_kph BETWEEN 5 AND 15 THEN 30  -- Offshore ideal
                WHEN w.wind_speed_kph <= 10 THEN 20  -- Vento fraco
                WHEN w.wind_speed_kph <= 20 THEN 10  -- Vento moderado  
                ELSE 0  -- Vento forte
            END) +
            -- Temperatura: 18-26°C é ideal
            (CASE 
                WHEN m.temperature_2m_celsius BETWEEN 18 AND 26 THEN 25
                WHEN m.temperature_2m_celsius BETWEEN 15 AND 30 THEN 15
                ELSE 5
            END) +
            -- Pressão: estável é melhor
            (CASE 
                WHEN m.pressure_msl_hpa BETWEEN 1013 AND 1023 THEN 25
                ELSE 10
            END) +
            -- Precipitação: sem chuva é melhor
            (CASE 
                WHEN m.precipitation_1h_mm = 0 THEN 20
                WHEN m.precipitation_1h_mm < 1 THEN 10
                ELSE 0
            END)
        ) as weather_score
        
    FROM silver_wind w
    INNER JOIN silver_meteomatics m 
        ON DATE_TRUNC('hour', w.datetime_utc) = DATE_TRUNC('hour', m.datetime_utc)
    WHERE w.wind_speed_kph IS NOT NULL 
      AND m.temperature_2m_celsius IS NOT NULL
),
unique_weather AS (
    SELECT DISTINCT
        wind_category,
        wind_direction_category,
        temperature_category,
        pressure_category,  
        precipitation_category,
        ROUND(AVG(weather_score), 1) as avg_weather_score
    FROM weather_combinations
    GROUP BY 1,2,3,4,5
)
SELECT 
    ROW_NUMBER() OVER (ORDER BY avg_weather_score DESC) as weather_key,
    wind_category,
    wind_direction_category,
    temperature_category,
    pressure_category,
    precipitation_category,
    avg_weather_score,
    
    -- Classificação geral do tempo
    CASE 
        WHEN avg_weather_score >= 70 THEN 'Condições Perfeitas'
        WHEN avg_weather_score >= 50 THEN 'Boas Condições'
        WHEN avg_weather_score >= 30 THEN 'Condições Regulares'
        ELSE 'Condições Ruins'
    END as weather_rating,
    
    -- Descrição combinada
    CONCAT(
        wind_category, ' ', wind_direction_category, ', ',
        temperature_category, ', ', precipitation_category
    ) as weather_description,
    
    -- Flag para condições ideais de surf
    CASE 
        WHEN wind_direction_category = 'Oeste' 
             AND wind_category IN ('Vento Fraco', 'Vento Moderado')
             AND temperature_category IN ('Agradável', 'Quente')
             AND precipitation_category = 'Sem Chuva'
        THEN true 
        ELSE false 
    END as is_ideal_surf_weather

FROM unique_weather
ORDER BY avg_weather_score DESC
"""

# Executa a query e cria a dimensão
dim_weather = con.execute(query_dim_weather).fetch_df()

print("🌤️ DIM_WEATHER CRIADA COM SUCESSO!")
print(f"Registros: {len(dim_weather):,}")
print("\nPrimeiras linhas:")
display(dim_weather.head(10))

print("\nDistribuição por rating do tempo:")
print(dim_weather['weather_rating'].value_counts())

print("\nCondições ideais para surf:")
ideal_conditions = dim_weather[dim_weather['is_ideal_surf_weather'] == True]
print(f"Combinações ideais: {len(ideal_conditions)}")
if len(ideal_conditions) > 0:
    display(ideal_conditions.head())

🌤️ DIM_WEATHER CRIADA COM SUCESSO!
Registros: 388

Primeiras linhas:


,weather_key,wind_category,wind_direction_category,temperature_category,pressure_category,precipitation_category,avg_weather_score,weather_rating,weather_description,is_ideal_surf_weather
0,1,Vento Fraco,Oeste,Agradável,Pressão Alta,Sem Chuva,95.0,Condições Perfeitas,"Vento Fraco Oeste, Agradável, Sem Chuva",True
1,2,Vento Fraco,Sul,Agradável,Pressão Alta,Sem Chuva,90.0,Condições Perfeitas,"Vento Fraco Sul, Agradável, Sem Chuva",False
2,3,Vento Fraco,Norte,Agradável,Pressão Alta,Sem Chuva,90.0,Condições Perfeitas,"Vento Fraco Norte, Agradável, Sem Chuva",False
3,4,Vento Moderado,Oeste,Fresco,Pressão Alta,Chuva Fraca,90.0,Condições Perfeitas,"Vento Moderado Oeste, Fresco, Chuva Fraca",False
4,5,Vento Fraco,Oeste,Agradável,Pressão Normal,Sem Chuva,87.5,Condições Perfeitas,"Vento Fraco Oeste, Agradável, Sem Chuva",True
5,6,Vento Fraco,Oeste,Fresco,Pressão Normal,Sem Chuva,86.0,Condições Perfeitas,"Vento Fraco Oeste, Fresco, Sem Chuva",False
6,7,Vento Fraco,Leste,Agradável,Pressão Normal,Sem Chuva,82.5,Condições Perfeitas,"Vento Fraco Leste, Agradável, Sem Chuva",False
7,8,Vento Fraco,Leste,Agradável,Pressão Alta,Sem Chuva,82.5,Condições Perfeitas,"Vento Fraco Leste, Agradável, Sem Chuva",False
8,9,Vento Moderado,Oeste,Agradável,Pressão Normal,Sem Chuva,82.5,Condições Perfeitas,"Vento Moderado Oeste, Agradável, Sem Chuva",True
9,10,Vento Fraco,Norte,Agradável,Pressão Normal,Sem Chuva,82.5,Condições Perfeitas,"Vento Fraco Norte, Agradável, Sem Chuva",False



Distribuição por rating do tempo:
weather_rating
Boas Condições         165
Condições Regulares    142
Condições Perfeitas     75
Condições Ruins          6
Name: count, dtype: int64

Condições ideais para surf:
Combinações ideais: 10


,weather_key,wind_category,wind_direction_category,temperature_category,pressure_category,precipitation_category,avg_weather_score,weather_rating,weather_description,is_ideal_surf_weather
0,1,Vento Fraco,Oeste,Agradável,Pressão Alta,Sem Chuva,95.0,Condições Perfeitas,"Vento Fraco Oeste, Agradável, Sem Chuva",True
4,5,Vento Fraco,Oeste,Agradável,Pressão Normal,Sem Chuva,87.5,Condições Perfeitas,"Vento Fraco Oeste, Agradável, Sem Chuva",True
8,9,Vento Moderado,Oeste,Agradável,Pressão Normal,Sem Chuva,82.5,Condições Perfeitas,"Vento Moderado Oeste, Agradável, Sem Chuva",True
10,11,Vento Fraco,Oeste,Quente,Pressão Normal,Sem Chuva,82.5,Condições Perfeitas,"Vento Fraco Oeste, Quente, Sem Chuva",True
15,16,Vento Fraco,Oeste,Agradável,Pressão Baixa,Sem Chuva,80.0,Condições Perfeitas,"Vento Fraco Oeste, Agradável, Sem Chuva",True


# 🎯 ETAPA 4: FACT_SURF_CONDITIONS - Tabela Fato Principal
## Consolidação de todas as métricas de surf por hora com chaves das dimensões

In [13]:
# ==============================================================================
# REGISTRANDO DIMENSÕES E CRIANDO FACT_SURF_CONDITIONS  
# ==============================================================================

# Registra as dimensões criadas no DuckDB
con.register('dim_date', dim_date)
con.register('dim_swell', dim_swell) 
con.register('dim_weather', dim_weather)

print("✅ DIMENSÕES REGISTRADAS NO DUCKDB")

# Query para criar a tabela fato principal
query_fact_surf = """
WITH base_fact AS (
    SELECT 
        -- Timestamp principal (granularidade horária)
        DATE_TRUNC('hour', s.datetime_utc) as datetime_utc,
        CAST(DATE_TRUNC('hour', s.datetime_utc) AS DATE) as date_utc,
        
        -- Métricas das ondas (tabela surf)
        s.wave_height_avg_m,
        s.wave_height_max_m,
        s.wave_height_min_m,
        s.wave_height_range_m,
        s.wave_size_category,
        
        -- Rating do surf
        r.rating_score,
        r.rating_category,
        
        -- Características dos swells
        sw.swell_1_height_m as primary_swell_height_m,
        sw.swell_1_period_s as primary_swell_period_s,
        sw.swell_1_direction_deg as primary_swell_direction_deg,
        sw.primary_swell_category,
        sw.primary_swell_direction,
        sw.total_swell_power,
        
        -- Condições da maré
        t.tide_height_m,
        t.tide_category,
        t.surf_condition as tide_surf_condition,
        t.tide_trend,
        
        -- Condições meteorológicas
        w.wind_speed_kph,
        w.wind_gust_kph,
        w.wind_direction_deg,
        m.temperature_2m_celsius,
        m.pressure_msl_hpa,
        m.precipitation_1h_mm,
        
        -- Luz solar
        sl.daylight_duration_hours,
        sl.surf_light_condition,
        sl.season
        
    FROM silver_surf s
    -- JOINs com as outras tabelas (usando timestamp truncado para hora)
    LEFT JOIN silver_rating r 
        ON DATE_TRUNC('hour', s.datetime_utc) = DATE_TRUNC('hour', r.datetime_utc)
    LEFT JOIN silver_swells sw 
        ON DATE_TRUNC('hour', s.datetime_utc) = DATE_TRUNC('hour', sw.datetime_utc)
    LEFT JOIN silver_tides t 
        ON DATE_TRUNC('hour', s.datetime_utc) = DATE_TRUNC('hour', t.datetime_utc)
    LEFT JOIN silver_wind w 
        ON DATE_TRUNC('hour', s.datetime_utc) = DATE_TRUNC('hour', w.datetime_utc)
    LEFT JOIN silver_meteomatics m 
        ON DATE_TRUNC('hour', s.datetime_utc) = DATE_TRUNC('hour', m.datetime_utc)
    LEFT JOIN silver_sunlight sl 
        ON CAST(s.datetime_utc AS DATE) = sl.date_local
        
    WHERE s.datetime_utc IS NOT NULL
),
fact_with_keys AS (
    SELECT 
        bf.*,
        
        -- Chaves das dimensões
        dd.date_key,
        
        -- Chave do swell (busca a combinação mais próxima)
        ds.swell_key,
        
        -- Chave do weather (busca a combinação mais próxima)  
        dw.weather_key
        
    FROM base_fact bf
    -- JOIN com dim_date
    LEFT JOIN dim_date dd ON bf.date_utc = dd.full_date
    
    -- JOIN com dim_swell (busca match exato das categorias)
    LEFT JOIN dim_swell ds ON (
        bf.primary_swell_category = ds.height_category 
        AND bf.primary_swell_direction = ds.direction_category
        AND CASE 
            WHEN bf.primary_swell_period_s < 8 THEN 'Período Curto'
            WHEN bf.primary_swell_period_s < 12 THEN 'Período Médio'  
            WHEN bf.primary_swell_period_s < 16 THEN 'Período Longo'
            ELSE 'Período Muito Longo'
        END = ds.period_category
    )
    
    -- JOIN com dim_weather (busca match das categorias)
    LEFT JOIN dim_weather dw ON (
        CASE 
            WHEN bf.wind_speed_kph <= 10 THEN 'Vento Fraco'
            WHEN bf.wind_speed_kph <= 20 THEN 'Vento Moderado'
            WHEN bf.wind_speed_kph <= 35 THEN 'Vento Forte'
            ELSE 'Vento Muito Forte'
        END = dw.wind_category
        AND
        CASE 
            WHEN bf.wind_direction_deg BETWEEN 0 AND 45 OR bf.wind_direction_deg BETWEEN 315 AND 360 THEN 'Norte'
            WHEN bf.wind_direction_deg BETWEEN 46 AND 135 THEN 'Leste'
            WHEN bf.wind_direction_deg BETWEEN 136 AND 225 THEN 'Sul'
            ELSE 'Oeste'
        END = dw.wind_direction_category
        AND
        CASE 
            WHEN bf.temperature_2m_celsius < 15 THEN 'Frio'
            WHEN bf.temperature_2m_celsius < 20 THEN 'Fresco'
            WHEN bf.temperature_2m_celsius < 25 THEN 'Agradável'
            WHEN bf.temperature_2m_celsius < 30 THEN 'Quente'
            ELSE 'Muito Quente'
        END = dw.temperature_category
    )
)
SELECT 
    -- Identificadores
    ROW_NUMBER() OVER (ORDER BY datetime_utc) as fact_id,
    datetime_utc,
    date_key,
    swell_key,
    weather_key,
    
    -- Métricas das ondas
    wave_height_avg_m,
    wave_height_max_m,
    wave_height_min_m,
    wave_height_range_m,
    
    -- Rating
    rating_score,
    
    -- Swells
    primary_swell_height_m,
    primary_swell_period_s, 
    total_swell_power,
    
    -- Maré
    tide_height_m,
    
    -- Meteorologia
    wind_speed_kph,
    wind_gust_kph,
    temperature_2m_celsius,
    pressure_msl_hpa,
    precipitation_1h_mm,
    
    -- Luz solar
    daylight_duration_hours,
    
    -- MÉTRICAS CALCULADAS (KPIs de Negócio)
    
    -- 1. Surf Quality Index (0-100)
    LEAST(100, 
        COALESCE(rating_score * 20, 0) +  -- 40% peso (score 1-5 -> *20 = 20-100)
        COALESCE(primary_swell_height_m * 15, 0) +   -- 30% peso  
        COALESCE(primary_swell_period_s * 2, 0) +    -- 20% peso
        CASE WHEN tide_surf_condition = 'Favorável' THEN 10 ELSE 0 END  -- 10% peso
    ) as surf_quality_index,
    
    -- 2. Session Potential Score (0-100)
    LEAST(100,
        COALESCE(wave_height_avg_m * 25, 0) +
        CASE WHEN wind_speed_kph <= 15 THEN 20 ELSE 5 END +
        CASE WHEN temperature_2m_celsius BETWEEN 18 AND 28 THEN 15 ELSE 5 END +
        CASE WHEN precipitation_1h_mm = 0 THEN 15 ELSE 0 END +
        CASE WHEN daylight_duration_hours > 10 THEN 10 ELSE 5 END
    ) as session_potential_score,
    
    -- 3. Flags úteis para análise
    CASE WHEN rating_score >= 4 THEN true ELSE false END as is_good_surf,
    CASE WHEN wave_height_avg_m >= 1.0 THEN true ELSE false END as is_surfable_size,
    CASE WHEN wind_speed_kph <= 20 THEN true ELSE false END as is_good_wind,
    CASE WHEN precipitation_1h_mm = 0 THEN true ELSE false END as is_no_rain
    
FROM fact_with_keys
WHERE datetime_utc IS NOT NULL
ORDER BY datetime_utc
"""

# Executa a query e cria a tabela fato
fact_surf_conditions = con.execute(query_fact_surf).fetch_df()

print("🎯 FACT_SURF_CONDITIONS CRIADA COM SUCESSO!")
print(f"Registros: {len(fact_surf_conditions):,}")
print(f"Período: {fact_surf_conditions['datetime_utc'].min()} até {fact_surf_conditions['datetime_utc'].max()}")
print("\nPrimeiras linhas:")
display(fact_surf_conditions.head())

✅ DIMENSÕES REGISTRADAS NO DUCKDB
🎯 FACT_SURF_CONDITIONS CRIADA COM SUCESSO!
Registros: 590,819
Período: 2020-01-01 03:00:00-03:00 até 2023-01-01 02:00:00-03:00

Primeiras linhas:
🎯 FACT_SURF_CONDITIONS CRIADA COM SUCESSO!
Registros: 590,819
Período: 2020-01-01 03:00:00-03:00 até 2023-01-01 02:00:00-03:00

Primeiras linhas:


,fact_id,datetime_utc,date_key,swell_key,weather_key,wave_height_avg_m,wave_height_max_m,wave_height_min_m,wave_height_range_m,rating_score,...,temperature_2m_celsius,pressure_msl_hpa,precipitation_1h_mm,daylight_duration_hours,surf_quality_index,session_potential_score,is_good_surf,is_surfable_size,is_good_wind,is_no_rain
0,1,2020-01-01 03:00:00-03:00,1,75,385,0.75,0.9,0.6,0.3,1,...,25.1,1006.0,0.05,13.846389,59.01915,48.75,False,False,False,False
1,2,2020-01-01 03:00:00-03:00,1,75,378,0.75,0.9,0.6,0.3,1,...,25.1,1006.0,0.05,13.846389,59.01915,48.75,False,False,False,False
2,3,2020-01-01 03:00:00-03:00,1,75,377,0.75,0.9,0.6,0.3,1,...,25.1,1006.0,0.05,13.846389,59.01915,48.75,False,False,False,False
3,4,2020-01-01 03:00:00-03:00,1,75,349,0.75,0.9,0.6,0.3,1,...,25.1,1006.0,0.05,13.846389,59.01915,48.75,False,False,False,False
4,5,2020-01-01 03:00:00-03:00,1,75,333,0.75,0.9,0.6,0.3,1,...,25.1,1006.0,0.05,13.846389,59.01915,48.75,False,False,False,False


# 💾 ETAPA 5: SALVANDO TABELAS DELTA GOLD
## Persistindo o Star Schema completo como Delta Tables

In [21]:
# ==============================================================================
# SALVANDO DIM_DATE COMO DELTA TABLE
# ==============================================================================

# Define o caminho para a Delta Table
gold_dim_date_path = os.path.join(GOLD_DIR, 'dim_date')

# Salva como Delta Table
write_deltalake(gold_dim_date_path, dim_date, mode='overwrite')

print(f"✅ DIM_DATE salva como Delta Table em: '{gold_dim_date_path}'")

# Verificação
dt_date_verification = DeltaTable(gold_dim_date_path).to_pandas()
print(f"📊 Verificação: {len(dt_date_verification):,} registros carregados")
print(f"🗓️ Período: {dt_date_verification['full_date'].min()} até {dt_date_verification['full_date'].max()}")
display(dt_date_verification.head(3))

✅ DIM_DATE salva como Delta Table em: 'data/gold/dim_date'
📊 Verificação: 1,097 registros carregados
🗓️ Período: 2020-01-01 00:00:00 até 2023-01-01 00:00:00


,date_key,full_date,year,month,day,quarter,week_of_year,day_of_week,day_name,month_name,is_weekend,season,surf_season,half_year,surf_period_description,year_month,year_quarter
0,1,2020-01-01,2020,1,1,1,1,3,Wednesday,January,False,Verão,Alta Temporada,Primeiro Semestre,Verão - Ondas Pequenas,2020-01,2020-Q1
1,2,2020-01-02,2020,1,2,1,1,4,Thursday,January,False,Verão,Alta Temporada,Primeiro Semestre,Verão - Ondas Pequenas,2020-01,2020-Q1
2,3,2020-01-03,2020,1,3,1,1,5,Friday,January,False,Verão,Alta Temporada,Primeiro Semestre,Verão - Ondas Pequenas,2020-01,2020-Q1


In [20]:
# ==============================================================================
# SALVANDO DIM_SWELL COMO DELTA TABLE
# ==============================================================================

# Define o caminho para a Delta Table
gold_dim_swell_path = os.path.join(GOLD_DIR, 'dim_swell')

# Salva como Delta Table
write_deltalake(gold_dim_swell_path, dim_swell, mode='overwrite')

print(f"✅ DIM_SWELL salva como Delta Table em: '{gold_dim_swell_path}'")

# Verificação
dt_swell_verification = DeltaTable(gold_dim_swell_path).to_pandas()
print(f"📊 Verificação: {len(dt_swell_verification):,} registros carregados")
print("\n🌊 Top 5 melhores swells:")
display(dt_swell_verification.head())

print("\nDistribuição por qualidade:")
print(dt_swell_verification['swell_quality'].value_counts())

✅ DIM_SWELL salva como Delta Table em: 'data/gold/dim_swell'
📊 Verificação: 83 registros carregados

🌊 Top 5 melhores swells:


,swell_key,height_category,direction_category,period_category,swell_quality,surf_potential,avg_swell_score,swell_description,simple_rating
0,1,Gigante,Sul,Período Médio,Swell Bom,Alto Potencial,100.0,Gigante - Sul - Período Médio,Bom para Surf
1,2,Gigante,Sul,Período Longo,Swell Épico,Alto Potencial,100.0,Gigante - Sul - Período Longo,Bom para Surf
2,3,Gigante,Sul,Período Médio,Swell Épico,Alto Potencial,100.0,Gigante - Sul - Período Médio,Bom para Surf
3,4,Gigante,Sul,Período Médio,Swell Bom,Médio Potencial,100.0,Gigante - Sul - Período Médio,Bom para Surf
4,5,Grande,Sul,Período Muito Longo,Swell Épico,Alto Potencial,100.0,Grande - Sul - Período Muito Longo,Bom para Surf



Distribuição por qualidade:
swell_quality
Swell Regular    32
Swell Bom        20
Swell Épico      17
Swell Fraco      14
Name: count, dtype: int64


In [22]:
# ==============================================================================
# SALVANDO DIM_WEATHER COMO DELTA TABLE
# ==============================================================================

# Define o caminho para a Delta Table
gold_dim_weather_path = os.path.join(GOLD_DIR, 'dim_weather')

# Salva como Delta Table
write_deltalake(gold_dim_weather_path, dim_weather, mode='overwrite')

print(f"✅ DIM_WEATHER salva como Delta Table em: '{gold_dim_weather_path}'")

# Verificação
dt_weather_verification = DeltaTable(gold_dim_weather_path).to_pandas()
print(f"📊 Verificação: {len(dt_weather_verification):,} registros carregados")
print("\n🌤️ Condições ideais para surf:")
ideal_weather = dt_weather_verification[dt_weather_verification['is_ideal_surf_weather'] == True]
display(ideal_weather[['weather_key', 'weather_description', 'avg_weather_score']].head())

print("\nDistribuição por rating do tempo:")
print(dt_weather_verification['weather_rating'].value_counts())

✅ DIM_WEATHER salva como Delta Table em: 'data/gold/dim_weather'
📊 Verificação: 388 registros carregados

🌤️ Condições ideais para surf:


,weather_key,weather_description,avg_weather_score
0,1,"Vento Fraco Oeste, Agradável, Sem Chuva",95.0
4,5,"Vento Fraco Oeste, Agradável, Sem Chuva",87.5
8,9,"Vento Moderado Oeste, Agradável, Sem Chuva",82.5
10,11,"Vento Fraco Oeste, Quente, Sem Chuva",82.5
15,16,"Vento Fraco Oeste, Agradável, Sem Chuva",80.0



Distribuição por rating do tempo:
weather_rating
Boas Condições         165
Condições Regulares    142
Condições Perfeitas     75
Condições Ruins          6
Name: count, dtype: int64


In [23]:
# ==============================================================================
# SALVANDO FACT_SURF_CONDITIONS COMO DELTA TABLE
# ==============================================================================

# Define o caminho para a Delta Table
gold_fact_surf_path = os.path.join(GOLD_DIR, 'fact_surf_conditions')

# Salva como Delta Table
write_deltalake(gold_fact_surf_path, fact_surf_conditions, mode='overwrite')

print(f"✅ FACT_SURF_CONDITIONS salva como Delta Table em: '{gold_fact_surf_path}'")

# Verificação
dt_fact_verification = DeltaTable(gold_fact_surf_path).to_pandas()
print(f"📊 Verificação: {len(dt_fact_verification):,} registros carregados")
print(f"🏄‍♂️ Período: {dt_fact_verification['datetime_utc'].min()} até {dt_fact_verification['datetime_utc'].max()}")

print("\n📈 MÉTRICAS DE NEGÓCIO:")
print(f"Surf Quality Index médio: {dt_fact_verification['surf_quality_index'].mean():.1f}")
print(f"Session Potential Score médio: {dt_fact_verification['session_potential_score'].mean():.1f}")

print("\n🏄‍♂️ FLAGS DE CONDIÇÕES:")
print(f"Bom para surf: {dt_fact_verification['is_good_surf'].sum():,} registros ({dt_fact_verification['is_good_surf'].mean()*100:.1f}%)")
print(f"Tamanho surfável: {dt_fact_verification['is_surfable_size'].sum():,} registros ({dt_fact_verification['is_surfable_size'].mean()*100:.1f}%)")
print(f"Vento bom: {dt_fact_verification['is_good_wind'].sum():,} registros ({dt_fact_verification['is_good_wind'].mean()*100:.1f}%)")
print(f"Sem chuva: {dt_fact_verification['is_no_rain'].sum():,} registros ({dt_fact_verification['is_no_rain'].mean()*100:.1f}%)")

print("\nPrimeiras linhas:")
display(dt_fact_verification.head())

✅ FACT_SURF_CONDITIONS salva como Delta Table em: 'data/gold/fact_surf_conditions'
📊 Verificação: 590,819 registros carregados
🏄‍♂️ Período: 2020-01-01 06:00:00+00:00 até 2023-01-01 05:00:00+00:00

📈 MÉTRICAS DE NEGÓCIO:
Surf Quality Index médio: 74.3
Session Potential Score médio: 69.2

🏄‍♂️ FLAGS DE CONDIÇÕES:
Bom para surf: 5,511 registros (0.9%)
Tamanho surfável: 334,850 registros (56.7%)
Vento bom: 165,000 registros (27.9%)
Sem chuva: 501,478 registros (84.9%)

Primeiras linhas:


,fact_id,datetime_utc,date_key,swell_key,weather_key,wave_height_avg_m,wave_height_max_m,wave_height_min_m,wave_height_range_m,rating_score,...,temperature_2m_celsius,pressure_msl_hpa,precipitation_1h_mm,daylight_duration_hours,surf_quality_index,session_potential_score,is_good_surf,is_surfable_size,is_good_wind,is_no_rain
0,1,2020-01-01 06:00:00+00:00,1,75,385,0.75,0.9,0.6,0.3,1,...,25.1,1006.0,0.05,13.846389,59.01915,48.75,False,False,False,False
1,2,2020-01-01 06:00:00+00:00,1,75,378,0.75,0.9,0.6,0.3,1,...,25.1,1006.0,0.05,13.846389,59.01915,48.75,False,False,False,False
2,3,2020-01-01 06:00:00+00:00,1,75,377,0.75,0.9,0.6,0.3,1,...,25.1,1006.0,0.05,13.846389,59.01915,48.75,False,False,False,False
3,4,2020-01-01 06:00:00+00:00,1,75,349,0.75,0.9,0.6,0.3,1,...,25.1,1006.0,0.05,13.846389,59.01915,48.75,False,False,False,False
4,5,2020-01-01 06:00:00+00:00,1,75,333,0.75,0.9,0.6,0.3,1,...,25.1,1006.0,0.05,13.846389,59.01915,48.75,False,False,False,False


In [24]:
# ==============================================================================
# 🏆 RESUMO FINAL DA CAMADA GOLD - STAR SCHEMA COMPLETO
# ==============================================================================

print("="*80)
print("🏆 CAMADA GOLD CRIADA COM SUCESSO!")
print("="*80)

print("\n📊 STAR SCHEMA - TABELAS DELTA CRIADAS:")
print("-" * 50)

# Verificar diretório Gold
import os
from pathlib import Path

gold_path = Path(GOLD_DIR)
if gold_path.exists():
    gold_tables = [d.name for d in gold_path.iterdir() if d.is_dir()]
    
    for table in sorted(gold_tables):
        table_path = gold_path / table
        print(f"✅ {table.upper()}")
        print(f"   📁 Caminho: {table_path}")
        
        # Carregar e mostrar informações básicas
        try:
            dt = DeltaTable(str(table_path))
            df = dt.to_pandas()
            print(f"   📊 Registros: {len(df):,}")
            
            if 'datetime_utc' in df.columns:
                print(f"   📅 Período: {df['datetime_utc'].min()} → {df['datetime_utc'].max()}")
            elif 'full_date' in df.columns:
                print(f"   📅 Período: {df['full_date'].min()} → {df['full_date'].max()}")
            
            print(f"   💾 Colunas: {len(df.columns)}")
            print()
        except Exception as e:
            print(f"   ❌ Erro ao ler: {e}")
            print()

print("\n🎯 ARQUITETURA STAR SCHEMA:")
print("-" * 50)
print("📐 DIMENSÕES:")
print("  🗓️  DIM_DATE     - Dimensão temporal (anos, meses, estações)")
print("  🌊 DIM_SWELL    - Dimensão de ondulação (altura, direção, período)")
print("  🌤️  DIM_WEATHER  - Dimensão meteorológica (vento, temperatura, chuva)")
print()
print("📊 FATOS:")
print("  🏄‍♂️ FACT_SURF_CONDITIONS - Condições de surf por hora com métricas de qualidade")

print("\n📈 MÉTRICAS DE NEGÓCIO IMPLEMENTADAS:")
print("-" * 50)
print("• Surf Quality Index - Índice geral de qualidade do surf")
print("• Session Potential Score - Potencial da sessão baseado em condições")
print("• Flags de condições (surf bom, tamanho surfável, vento bom, sem chuva)")
print("• Categorização de swells por qualidade e potencial")
print("• Análise temporal com estações de surf")

print("\n🚀 PRÓXIMOS PASSOS SUGERIDOS:")
print("-" * 50)
print("1. 📊 Criar dashboards com Power BI/Tableau")
print("2. 🤖 Implementar modelos de ML para previsão")
print("3. 📱 Desenvolver aplicativo para surfistas")
print("4. 🔄 Automatizar pipeline com Airflow")
print("5. ⚡ Criar APIs para consulta em tempo real")

print("\n" + "="*80)
print("🎉 GOLD LAYER COMPLETADO - PRONTO PARA ANALYTICS!")
print("="*80)

🏆 CAMADA GOLD CRIADA COM SUCESSO!

📊 STAR SCHEMA - TABELAS DELTA CRIADAS:
--------------------------------------------------
✅ DIM_DATE
   📁 Caminho: data/gold/dim_date
   📊 Registros: 1,097
   📅 Período: 2020-01-01 00:00:00 → 2023-01-01 00:00:00
   💾 Colunas: 17

✅ DIM_SWELL
   📁 Caminho: data/gold/dim_swell
   📊 Registros: 83
   💾 Colunas: 9

✅ DIM_WEATHER
   📁 Caminho: data/gold/dim_weather
   📊 Registros: 388
   💾 Colunas: 10

✅ FACT_SURF_CONDITIONS
   📁 Caminho: data/gold/fact_surf_conditions
   📊 Registros: 590,819
   📅 Período: 2020-01-01 06:00:00+00:00 → 2023-01-01 05:00:00+00:00
   💾 Colunas: 26


🎯 ARQUITETURA STAR SCHEMA:
--------------------------------------------------
📐 DIMENSÕES:
  🗓️  DIM_DATE     - Dimensão temporal (anos, meses, estações)
  🌊 DIM_SWELL    - Dimensão de ondulação (altura, direção, período)
  🌤️  DIM_WEATHER  - Dimensão meteorológica (vento, temperatura, chuva)

📊 FATOS:
  🏄‍♂️ FACT_SURF_CONDITIONS - Condições de surf por hora com métricas de qualidade

# 📊 ETAPA 6: DATA MARTS E ANÁLISES DE NEGÓCIO
## Criando tabelas agregadas especializadas para insights específicos

### 🎯 **Data Marts Planejados:**
1. **MART_BEST_SURF_TIMES** - Melhores horários para surfar por período
2. **MART_SWELL_ANALYSIS** - Análise detalhada de swells e sua qualidade
3. **MART_WEATHER_PATTERNS** - Padrões climáticos ideais para surf
4. **MART_SEASONAL_TRENDS** - Tendências sazonais de condições de surf
5. **MART_FORECAST_MODEL** - Dados preparados para modelo de previsão
6. **MART_SPOT_PERFORMANCE** - Performance do spot por condições
7. **MART_SESSION_RECOMMENDATIONS** - Recomendações de sessão personalizadas

### 🧠 **Perguntas de Negócio para Responder:**
- Qual o melhor horário do dia para surfar?
- Quais meses têm as melhores condições?
- Como a chuva realmente afeta as condições? (Sua observação está certa!)
- Qual a relação entre vento e qualidade das ondas?
- Quando vale a pena acordar cedo para surfar?
- Quais combinações de swell produzem as melhores sessões?

In [25]:
# ==============================================================================
# MART 1: BEST_SURF_TIMES - Melhores Horários para Surfar
# ==============================================================================

query_mart_best_times = """
WITH hourly_stats AS (
    SELECT 
        EXTRACT(HOUR FROM datetime_utc) as hour_of_day,
        
        -- Métricas de qualidade
        AVG(surf_quality_index) as avg_surf_quality,
        AVG(session_potential_score) as avg_session_potential,
        AVG(rating_score) as avg_rating,
        AVG(wave_height_avg_m) as avg_wave_height,
        
        -- Condições climáticas
        AVG(wind_speed_kph) as avg_wind_speed,
        AVG(temperature_2m_celsius) as avg_temperature,
        
        -- Percentuais de boas condições
        AVG(CASE WHEN is_good_surf THEN 1.0 ELSE 0.0 END) * 100 as pct_good_surf,
        AVG(CASE WHEN is_surfable_size THEN 1.0 ELSE 0.0 END) * 100 as pct_surfable_size,
        AVG(CASE WHEN is_good_wind THEN 1.0 ELSE 0.0 END) * 100 as pct_good_wind,
        AVG(CASE WHEN is_no_rain THEN 1.0 ELSE 0.0 END) * 100 as pct_no_rain,
        
        -- Contadores
        COUNT(*) as total_observations,
        COUNT(CASE WHEN is_good_surf THEN 1 END) as good_surf_sessions,
        
        -- Análise de luz solar
        AVG(daylight_duration_hours) as avg_daylight_hours
        
    FROM fact_surf_conditions
    WHERE datetime_utc IS NOT NULL
    GROUP BY EXTRACT(HOUR FROM datetime_utc)
),
ranked_hours AS (
    SELECT *,
        -- Ranking por diferentes critérios
        RANK() OVER (ORDER BY avg_surf_quality DESC) as rank_by_quality,
        RANK() OVER (ORDER BY pct_good_surf DESC) as rank_by_good_surf_pct,
        RANK() OVER (ORDER BY avg_session_potential DESC) as rank_by_session_potential,
        
        -- Score combinado (você pode ajustar os pesos)
        (avg_surf_quality * 0.4 + 
         avg_session_potential * 0.3 + 
         pct_good_surf * 0.2 + 
         pct_surfable_size * 0.1) as combined_score,
         
        -- Classificação do horário
        CASE 
            WHEN hour_of_day BETWEEN 5 AND 8 THEN 'Dawn Patrol (Madrugada)'
            WHEN hour_of_day BETWEEN 9 AND 11 THEN 'Manhã'
            WHEN hour_of_day BETWEEN 12 AND 14 THEN 'Meio-dia'
            WHEN hour_of_day BETWEEN 15 AND 17 THEN 'Tarde'
            WHEN hour_of_day BETWEEN 18 AND 20 THEN 'Final de Tarde'
            ELSE 'Noite/Madrugada'
        END as time_period
        
    FROM hourly_stats
)
SELECT 
    hour_of_day,
    time_period,
    ROUND(avg_surf_quality, 1) as avg_surf_quality,
    ROUND(avg_session_potential, 1) as avg_session_potential,
    ROUND(pct_good_surf, 1) as pct_good_surf,
    ROUND(pct_surfable_size, 1) as pct_surfable_size,
    ROUND(avg_wave_height, 2) as avg_wave_height,
    ROUND(avg_wind_speed, 1) as avg_wind_speed,
    ROUND(avg_temperature, 1) as avg_temperature,
    total_observations,
    good_surf_sessions,
    rank_by_quality,
    ROUND(combined_score, 1) as combined_score,
    
    -- Recomendação
    CASE 
        WHEN combined_score >= 60 THEN '🌟 Horário Premium'
        WHEN combined_score >= 45 THEN '🏄‍♂️ Bom para Surf'
        WHEN combined_score >= 30 THEN '⚡ Surf Médio'
        ELSE '😴 Melhor Descansar'
    END as recommendation

FROM ranked_hours
ORDER BY combined_score DESC
"""

# Executa e cria o mart
mart_best_surf_times = con.execute(query_mart_best_times).fetch_df()

print("🕐 MART_BEST_SURF_TIMES CRIADO COM SUCESSO!")
print(f"Registros: {len(mart_best_surf_times)}")
print("\n🏆 TOP 5 MELHORES HORÁRIOS PARA SURFAR:")
display(mart_best_surf_times.head())

print("\n📊 ANÁLISE POR PERÍODO DO DIA:")
period_analysis = mart_best_surf_times.groupby('time_period').agg({
    'avg_surf_quality': 'mean',
    'pct_good_surf': 'mean',
    'avg_wave_height': 'mean',
    'total_observations': 'sum'
}).round(1)
display(period_analysis)

🕐 MART_BEST_SURF_TIMES CRIADO COM SUCESSO!
Registros: 24

🏆 TOP 5 MELHORES HORÁRIOS PARA SURFAR:


,hour_of_day,time_period,avg_surf_quality,avg_session_potential,pct_good_surf,pct_surfable_size,avg_wave_height,avg_wind_speed,avg_temperature,total_observations,good_surf_sessions,rank_by_quality,combined_score,recommendation
0,9,Manhã,76.2,70.3,1.5,56.6,1.06,25.8,20.9,23858,349,4,57.5,🏄‍♂️ Bom para Surf
1,8,Dawn Patrol (Madrugada),76.5,70.1,1.4,55.9,1.06,25.5,20.5,23967,338,1,57.5,🏄‍♂️ Bom para Surf
2,15,Tarde,75.9,70.0,1.2,59.0,1.07,26.2,21.8,23954,295,6,57.5,🏄‍♂️ Bom para Surf
3,16,Tarde,76.0,69.7,1.3,58.5,1.07,26.2,21.6,23559,298,5,57.4,🏄‍♂️ Bom para Surf
4,11,Manhã,75.1,70.8,1.4,57.6,1.07,25.5,21.8,23377,328,9,57.3,🏄‍♂️ Bom para Surf



📊 ANÁLISE POR PERÍODO DO DIA:


,avg_surf_quality,pct_good_surf,avg_wave_height,total_observations
time_period,,,,
Dawn Patrol (Madrugada),76.1,1.1,1.1,97042
Final de Tarde,74.0,0.6,1.1,73884
Manhã,75.6,1.4,1.1,71016
Meio-dia,74.5,1.7,1.1,72794
Noite/Madrugada,72.4,0.4,1.1,204178
Tarde,75.8,1.3,1.1,71905


In [26]:
# ==============================================================================
# MART 2: RAIN_ANALYSIS - Análise Especial sobre Chuva e Surf
# (Testando sua teoria de que chuva pode ser boa para surf!)
# ==============================================================================

query_mart_rain_analysis = """
WITH rain_categories AS (
    SELECT 
        CASE 
            WHEN precipitation_1h_mm = 0 THEN 'Sem Chuva'
            WHEN precipitation_1h_mm <= 1 THEN 'Chuva Leve (0-1mm)'
            WHEN precipitation_1h_mm <= 5 THEN 'Chuva Moderada (1-5mm)'
            WHEN precipitation_1h_mm <= 10 THEN 'Chuva Forte (5-10mm)'
            ELSE 'Chuva Intensa (>10mm)'
        END as rain_category,
        
        precipitation_1h_mm,
        wind_speed_kph,
        surf_quality_index,
        session_potential_score,
        rating_score,
        wave_height_avg_m,
        temperature_2m_celsius,
        
        -- Flags
        is_good_surf,
        is_surfable_size,
        is_good_wind,
        
        -- Análise de vento durante chuva
        CASE 
            WHEN precipitation_1h_mm > 0 AND wind_speed_kph <= 15 THEN 'Chuva + Vento Fraco'
            WHEN precipitation_1h_mm > 0 AND wind_speed_kph > 15 THEN 'Chuva + Vento Forte'
            ELSE 'Sem Chuva'
        END as rain_wind_combo
        
    FROM fact_surf_conditions
    WHERE precipitation_1h_mm IS NOT NULL
),
rain_stats AS (
    SELECT 
        rain_category,
        
        -- Estatísticas gerais
        COUNT(*) as total_sessions,
        AVG(surf_quality_index) as avg_surf_quality,
        AVG(session_potential_score) as avg_session_potential,
        AVG(rating_score) as avg_rating,
        AVG(wave_height_avg_m) as avg_wave_height,
        AVG(wind_speed_kph) as avg_wind_speed,
        AVG(temperature_2m_celsius) as avg_temperature,
        
        -- Percentuais de boas condições
        AVG(CASE WHEN is_good_surf THEN 1.0 ELSE 0.0 END) * 100 as pct_good_surf,
        AVG(CASE WHEN is_surfable_size THEN 1.0 ELSE 0.0 END) * 100 as pct_surfable_size,
        AVG(CASE WHEN is_good_wind THEN 1.0 ELSE 0.0 END) * 100 as pct_good_wind,
        
        -- Análise específica: surf bom EM DIAS DE CHUVA
        COUNT(CASE WHEN is_good_surf THEN 1 END) as good_surf_rainy_sessions
        
    FROM rain_categories
    GROUP BY rain_category
)
SELECT 
    rain_category,
    total_sessions,
    ROUND(avg_surf_quality, 1) as avg_surf_quality,
    ROUND(avg_session_potential, 1) as avg_session_potential,
    ROUND(avg_rating, 2) as avg_rating,
    ROUND(avg_wave_height, 2) as avg_wave_height,
    ROUND(avg_wind_speed, 1) as avg_wind_speed,
    ROUND(pct_good_surf, 1) as pct_good_surf,
    ROUND(pct_good_wind, 1) as pct_good_wind,
    good_surf_rainy_sessions,
    
    -- Ranking de qualidade
    RANK() OVER (ORDER BY avg_surf_quality DESC) as quality_rank,
    
    -- Insight: chuva vs sem chuva
    CASE 
        WHEN avg_surf_quality >= 70 AND rain_category != 'Sem Chuva' THEN '🌧️💎 Chuva Boa para Surf!'
        WHEN avg_surf_quality >= 70 THEN '☀️ Clássico Sem Chuva'
        WHEN avg_surf_quality >= 60 THEN '⚡ Surfável'
        ELSE '😬 Condições Ruins'
    END as rain_insight

FROM rain_stats
ORDER BY avg_surf_quality DESC
"""

# Executa e cria o mart
mart_rain_analysis = con.execute(query_mart_rain_analysis).fetch_df()

print("🌧️ MART_RAIN_ANALYSIS CRIADO COM SUCESSO!")
print("🔍 TESTANDO A TEORIA: 'CHUVA PODE SER BOA PARA SURF'")
print(f"Registros: {len(mart_rain_analysis)}")
display(mart_rain_analysis)

print("\n💡 INSIGHTS SOBRE CHUVA E SURF:")
print("=" * 50)

# Análise comparativa
sem_chuva = mart_rain_analysis[mart_rain_analysis['rain_category'] == 'Sem Chuva'].iloc[0] if len(mart_rain_analysis[mart_rain_analysis['rain_category'] == 'Sem Chuva']) > 0 else None
com_chuva = mart_rain_analysis[mart_rain_analysis['rain_category'] != 'Sem Chuva']

if sem_chuva is not None and len(com_chuva) > 0:
    print(f"📊 Qualidade média SEM chuva: {sem_chuva['avg_surf_quality']}")
    print(f"🌧️ Melhor qualidade COM chuva: {com_chuva['avg_surf_quality'].max()}")
    print(f"💨 Vento médio SEM chuva: {sem_chuva['avg_wind_speed']} km/h")
    print(f"🌧️💨 Vento médio COM chuva: {com_chuva['avg_wind_speed'].mean():.1f} km/h")
    
    if com_chuva['avg_wind_speed'].mean() < sem_chuva['avg_wind_speed']:
        print("\n✅ TEORIA CONFIRMADA: Chuva = Menos Vento = Melhores Condições!")
    else:
        print("\n❓ Teoria parcialmente confirmada - precisa mais análise...")

🌧️ MART_RAIN_ANALYSIS CRIADO COM SUCESSO!
🔍 TESTANDO A TEORIA: 'CHUVA PODE SER BOA PARA SURF'
Registros: 5


,rain_category,total_sessions,avg_surf_quality,avg_session_potential,avg_rating,avg_wave_height,avg_wind_speed,pct_good_surf,pct_good_wind,good_surf_rainy_sessions,quality_rank,rain_insight
0,Sem Chuva,501478,74.7,71.6,1.52,1.07,25.9,1.0,28.8,4963,1,☀️ Clássico Sem Chuva
1,Chuva Forte (5-10mm),5395,72.4,57.3,1.24,1.13,28.1,0.0,25.5,0,2,🌧️💎 Chuva Boa para Surf!
2,Chuva Moderada (1-5mm),32274,72.4,55.7,1.35,1.06,27.7,0.2,25.1,79,3,🌧️💎 Chuva Boa para Surf!
3,Chuva Leve (0-1mm),50204,71.7,55.3,1.37,1.02,28.1,0.9,21.3,469,4,🌧️💎 Chuva Boa para Surf!
4,Chuva Intensa (>10mm),1454,68.2,49.4,1.40,0.78,26.5,0.0,27.2,0,5,⚡ Surfável



💡 INSIGHTS SOBRE CHUVA E SURF:
📊 Qualidade média SEM chuva: 74.7
🌧️ Melhor qualidade COM chuva: 72.4
💨 Vento médio SEM chuva: 25.9 km/h
🌧️💨 Vento médio COM chuva: 27.6 km/h

❓ Teoria parcialmente confirmada - precisa mais análise...


In [30]:
# ==============================================================================
# MART 3: SWELL_PATTERNS - Análise de Padrões de Swell
# ==============================================================================

query_mart_swell_patterns = """
WITH swell_analysis AS (
    SELECT 
        f.*,
        ds.height_category,
        ds.direction_category,
        ds.period_category,
        ds.swell_quality,
        ds.surf_potential,
        ds.swell_description,
        
        -- Análise temporal
        EXTRACT(MONTH FROM f.datetime_utc) as month,
        EXTRACT(HOUR FROM f.datetime_utc) as hour,
        
        -- Classificação de swell combinado
        CASE 
            WHEN ds.swell_quality = 'Swell Épico' AND f.rating_score >= 4 THEN 'Sessão Épica'
            WHEN ds.swell_quality = 'Swell Bom' AND f.rating_score >= 3 THEN 'Sessão Boa'
            WHEN ds.swell_quality = 'Swell Regular' THEN 'Sessão Regular'
            ELSE 'Sessão Fraca'
        END as session_classification
        
    FROM fact_surf_conditions f
    LEFT JOIN dim_swell ds ON f.swell_key = ds.swell_key
    WHERE f.swell_key IS NOT NULL
),
swell_stats AS (
    SELECT 
        height_category,
        direction_category,
        period_category,
        swell_quality,
        session_classification,
        
        -- Estatísticas básicas
        COUNT(*) as total_occurrences,
        AVG(surf_quality_index) as avg_surf_quality,
        AVG(session_potential_score) as avg_session_potential,
        AVG(wave_height_avg_m) as avg_wave_height,
        AVG(rating_score) as avg_rating,
        
        -- Melhores condições
        MAX(surf_quality_index) as max_surf_quality,
        COUNT(CASE WHEN is_good_surf THEN 1 END) as epic_sessions,
        
        -- Análise temporal - melhores meses
        STRING_AGG(DISTINCT CAST(month AS VARCHAR), ', ') as best_months,
        
        -- Probabilidade de surf bom
        AVG(CASE WHEN is_good_surf THEN 1.0 ELSE 0.0 END) * 100 as success_rate
        
    FROM swell_analysis
    GROUP BY height_category, direction_category, period_category, swell_quality, session_classification
)
SELECT 
    height_category,
    direction_category, 
    period_category,
    swell_quality,
    session_classification,
    total_occurrences,
    ROUND(avg_surf_quality, 1) as avg_surf_quality,
    ROUND(avg_session_potential, 1) as avg_session_potential,
    ROUND(avg_wave_height, 2) as avg_wave_height,
    ROUND(success_rate, 1) as success_rate_pct,
    epic_sessions,
    
    -- Score de preferência (combinação de qualidade + frequência)
    ROUND(avg_surf_quality * LOG(total_occurrences + 1), 1) as preference_score,
    
    -- Recomendação
    CASE 
        WHEN avg_surf_quality >= 80 AND success_rate >= 5 THEN '🏆 HUNT THIS SWELL!'
        WHEN avg_surf_quality >= 70 THEN '🎯 Swell Confiável'
        WHEN avg_surf_quality >= 60 THEN '⚡ Swell OK'
        ELSE '😴 Skip'
    END as recommendation

FROM swell_stats
WHERE total_occurrences >= 10  -- Filtra swells com dados suficientes
ORDER BY preference_score DESC
"""

# Executa e cria o mart
mart_swell_patterns = con.execute(query_mart_swell_patterns).fetch_df()

print("🌊 MART_SWELL_PATTERNS CRIADO COM SUCESSO!")
print(f"Registros: {len(mart_swell_patterns)}")
print("\n🏆 TOP 10 MELHORES PADRÕES DE SWELL:")
display(mart_swell_patterns.head(10))

print("\n📊 ANÁLISE POR DIREÇÃO DE SWELL:")
direction_analysis = mart_swell_patterns.groupby('direction_category').agg({
    'avg_surf_quality': 'mean',
    'success_rate_pct': 'mean',
    'total_occurrences': 'sum',
    'epic_sessions': 'sum'
}).round(1).sort_values('avg_surf_quality', ascending=False)
display(direction_analysis)

print("\n🎯 SWELLS PARA 'HUNTAR' (>80 qualidade + >5% sucesso):")
hunt_swells = mart_swell_patterns[mart_swell_patterns['recommendation'] == '🏆 HUNT THIS SWELL!']
if len(hunt_swells) > 0:
    display(hunt_swells[['height_category', 'direction_category', 'period_category', 'avg_surf_quality', 'success_rate_pct', 'epic_sessions']])
else:
    print("Nenhum swell épico encontrado com critérios rigorosos - vamos relaxar os critérios!")

🌊 MART_SWELL_PATTERNS CRIADO COM SUCESSO!
Registros: 84

🏆 TOP 10 MELHORES PADRÕES DE SWELL:


,height_category,direction_category,period_category,swell_quality,session_classification,total_occurrences,avg_surf_quality,avg_session_potential,avg_wave_height,success_rate_pct,epic_sessions,preference_score,recommendation
0,Médio,Sul,Período Médio,Swell Bom,Sessão Boa,18908,99.7,77.5,1.14,5.4,1016,426.3,🏆 HUNT THIS SWELL!
1,Médio,Leste,Período Médio,Swell Bom,Sessão Boa,9973,99.5,74.7,1.03,2.8,278,397.7,🎯 Swell Confiável
2,Grande,Sul,Período Médio,Swell Bom,Sessão Boa,8612,100.0,84.3,1.54,10.1,874,393.5,🏆 HUNT THIS SWELL!
3,Médio,Sul,Período Longo,Swell Bom,Sessão Boa,7164,100.0,77.3,1.31,8.6,618,385.5,🏆 HUNT THIS SWELL!
4,Grande,Leste,Período Médio,Swell Bom,Sessão Boa,4670,100.0,78.7,1.26,14.1,660,366.9,🏆 HUNT THIS SWELL!
5,Médio,Sul,Período Médio,Swell Bom,Sessão Fraca,45410,78.7,69.4,1.14,0.0,0,366.4,🎯 Swell Confiável
6,Grande,Sul,Período Longo,Swell Épico,Sessão Fraca,6995,95.0,84.6,1.68,0.0,0,365.2,🎯 Swell Confiável
7,Grande,Leste,Período Médio,Swell Épico,Sessão Fraca,21840,83.2,72.7,1.20,0.0,0,361.2,🎯 Swell Confiável
8,Grande,Sul,Período Médio,Swell Épico,Sessão Fraca,16812,85.5,76.9,1.44,0.0,0,361.1,🎯 Swell Confiável
9,Grande,Sul,Período Médio,Swell Bom,Sessão Fraca,25886,81.1,74.7,1.40,0.0,0,357.9,🎯 Swell Confiável



📊 ANÁLISE POR DIREÇÃO DE SWELL:


,avg_surf_quality,success_rate_pct,total_occurrences,epic_sessions
direction_category,,,,
Sul,81.5,12.3,267764,3526
Leste,81.4,14.6,295345,1985
Norte,78.4,0.0,27580,0
Oeste,62.7,0.0,130,0



🎯 SWELLS PARA 'HUNTAR' (>80 qualidade + >5% sucesso):


,height_category,direction_category,period_category,avg_surf_quality,success_rate_pct,epic_sessions
0,Médio,Sul,Período Médio,99.7,5.4,1016
2,Grande,Sul,Período Médio,100.0,10.1,874
3,Médio,Sul,Período Longo,100.0,8.6,618
4,Grande,Leste,Período Médio,100.0,14.1,660
22,Muito Grande,Sul,Período Médio,100.0,5.5,54
23,Médio,Leste,Período Longo,100.0,19.9,188
27,Grande,Leste,Período Médio,100.0,100.0,660
30,Grande,Sul,Período Médio,100.0,100.0,437
33,Grande,Sul,Período Longo,100.0,100.0,399
49,Muito Grande,Sul,Período Longo,100.0,100.0,101


In [28]:
# ==============================================================================
# MART 4: SEASONAL_INSIGHTS - Análise Sazonal Detalhada
# ==============================================================================

query_mart_seasonal = """
WITH seasonal_data AS (
    SELECT 
        f.*,
        dd.season,
        dd.surf_season,
        dd.month,
        dd.month_name,
        dd.is_weekend,
        dd.surf_period_description,
        
        -- Análise por quartil do ano
        CASE 
            WHEN dd.month IN (12, 1, 2) THEN 'Q4-Q1: Verão'
            WHEN dd.month IN (3, 4, 5) THEN 'Q2: Outono'  
            WHEN dd.month IN (6, 7, 8) THEN 'Q3: Inverno'
            ELSE 'Q4: Primavera'
        END as quarter_season,
        
        -- Classificação de fim de semana
        CASE 
            WHEN dd.is_weekend THEN 'Fim de Semana'
            ELSE 'Semana'
        END as weekend_category
        
    FROM fact_surf_conditions f
    LEFT JOIN dim_date dd ON f.date_key = dd.date_key
    WHERE f.date_key IS NOT NULL
),
seasonal_stats AS (
    SELECT 
        month,
        month_name,
        season,
        surf_season,
        quarter_season,
        surf_period_description,
        
        -- Estatísticas básicas
        COUNT(*) as total_sessions,
        AVG(surf_quality_index) as avg_surf_quality,
        AVG(session_potential_score) as avg_session_potential,
        AVG(wave_height_avg_m) as avg_wave_height,
        AVG(wind_speed_kph) as avg_wind_speed,
        AVG(temperature_2m_celsius) as avg_temperature,
        
        -- Análise de condições ideais
        COUNT(CASE WHEN is_good_surf THEN 1 END) as good_surf_days,
        AVG(CASE WHEN is_good_surf THEN 1.0 ELSE 0.0 END) * 100 as pct_good_surf,
        AVG(CASE WHEN is_surfable_size THEN 1.0 ELSE 0.0 END) * 100 as pct_surfable,
        AVG(CASE WHEN is_good_wind THEN 1.0 ELSE 0.0 END) * 100 as pct_good_wind,
        AVG(CASE WHEN is_no_rain THEN 1.0 ELSE 0.0 END) * 100 as pct_no_rain,
        
        -- Métricas extremas
        MAX(surf_quality_index) as best_session_quality,
        MIN(surf_quality_index) as worst_session_quality,
        MAX(wave_height_avg_m) as biggest_waves,
        
        -- Análise de consistência
        STDDEV(surf_quality_index) as quality_consistency
        
    FROM seasonal_data
    GROUP BY month, month_name, season, surf_season, quarter_season, surf_period_description
)
SELECT 
    month,
    month_name,
    season,
    surf_season,
    surf_period_description,
    total_sessions,
    ROUND(avg_surf_quality, 1) as avg_surf_quality,
    ROUND(avg_session_potential, 1) as avg_session_potential,
    ROUND(avg_wave_height, 2) as avg_wave_height,
    ROUND(avg_wind_speed, 1) as avg_wind_speed,
    ROUND(avg_temperature, 1) as avg_temperature,
    good_surf_days,
    ROUND(pct_good_surf, 1) as pct_good_surf,
    ROUND(pct_surfable, 1) as pct_surfable,
    ROUND(biggest_waves, 2) as biggest_waves,
    ROUND(quality_consistency, 1) as consistency_score,
    
    -- Rankings
    RANK() OVER (ORDER BY avg_surf_quality DESC) as quality_rank,
    RANK() OVER (ORDER BY pct_good_surf DESC) as success_rank,
    RANK() OVER (ORDER BY biggest_waves DESC) as size_rank,
    
    -- Recomendações sazonais
    CASE 
        WHEN avg_surf_quality >= 75 AND pct_good_surf >= 2 THEN '🏆 Mês Premium'
        WHEN avg_surf_quality >= 70 THEN '🌟 Mês Bom'
        WHEN avg_surf_quality >= 65 THEN '⚡ Mês OK'
        ELSE '😴 Mês Fraco'
    END as month_rating,
    
    -- Estratégia do mês
    CASE 
        WHEN biggest_waves >= 2.0 THEN '🌊 Foque em Ondas Grandes'
        WHEN pct_good_surf >= 2 THEN '🎯 Aproveite a Consistência'
        WHEN avg_temperature >= 25 THEN '☀️ Mês para Relaxar na Água'
        ELSE '🔍 Seja Seletivo'
    END as monthly_strategy

FROM seasonal_stats
ORDER BY avg_surf_quality DESC
"""

# Executa e cria o mart
mart_seasonal_insights = con.execute(query_mart_seasonal).fetch_df()

print("🗓️ MART_SEASONAL_INSIGHTS CRIADO COM SUCESSO!")
print(f"Registros: {len(mart_seasonal_insights)}")
print("\n🏆 RANKING DOS MELHORES MESES PARA SURFAR:")
display(mart_seasonal_insights[['month_name', 'avg_surf_quality', 'pct_good_surf', 'biggest_waves', 'month_rating', 'monthly_strategy']].head(12))

print("\n📊 ANÁLISE POR ESTAÇÃO:")
season_summary = mart_seasonal_insights.groupby('season').agg({
    'avg_surf_quality': 'mean',
    'pct_good_surf': 'mean', 
    'avg_wave_height': 'mean',
    'biggest_waves': 'max',
    'total_sessions': 'sum'
}).round(1).sort_values('avg_surf_quality', ascending=False)
display(season_summary)

print("\n🎯 MESES PREMIUM (>75 qualidade + >2% sucesso):")
premium_months = mart_seasonal_insights[mart_seasonal_insights['month_rating'] == '🏆 Mês Premium']
if len(premium_months) > 0:
    display(premium_months[['month_name', 'avg_surf_quality', 'pct_good_surf', 'biggest_waves', 'monthly_strategy']])
else:
    print("Nenhum mês atingiu critérios premium - vamos ver os melhores disponíveis:")
    display(mart_seasonal_insights.head(3)[['month_name', 'avg_surf_quality', 'pct_good_surf', 'monthly_strategy']])

🗓️ MART_SEASONAL_INSIGHTS CRIADO COM SUCESSO!
Registros: 12

🏆 RANKING DOS MELHORES MESES PARA SURFAR:


,month_name,avg_surf_quality,pct_good_surf,biggest_waves,month_rating,monthly_strategy
0,September,79.1,1.7,2.40,🌟 Mês Bom,🌊 Foque em Ondas Grandes
1,May,78.7,4.3,3.05,🏆 Mês Premium,🌊 Foque em Ondas Grandes
2,April,78.0,0.9,3.05,🌟 Mês Bom,🌊 Foque em Ondas Grandes
3,August,76.7,0.6,3.05,🌟 Mês Bom,🌊 Foque em Ondas Grandes
4,July,76.3,0.7,3.05,🌟 Mês Bom,🌊 Foque em Ondas Grandes
5,June,76.3,0.8,2.40,🌟 Mês Bom,🌊 Foque em Ondas Grandes
6,November,73.1,0.0,2.40,🌟 Mês Bom,🌊 Foque em Ondas Grandes
7,October,71.4,0.7,1.95,🌟 Mês Bom,🔍 Seja Seletivo
8,February,71.1,0.0,2.40,🌟 Mês Bom,🌊 Foque em Ondas Grandes
9,January,71.0,0.4,2.40,🌟 Mês Bom,🌊 Foque em Ondas Grandes



📊 ANÁLISE POR ESTAÇÃO:


,avg_surf_quality,pct_good_surf,avg_wave_height,biggest_waves,total_sessions
season,,,,,
Inverno,76.4,0.7,1.1,3.0,147569
Outono,75.5,1.9,1.1,3.0,151028
Primavera,74.5,0.8,1.1,2.4,148391
Verão,70.5,0.2,0.9,2.4,143831



🎯 MESES PREMIUM (>75 qualidade + >2% sucesso):


,month_name,avg_surf_quality,pct_good_surf,biggest_waves,monthly_strategy
1,May,78.7,4.3,3.05,🌊 Foque em Ondas Grandes


In [31]:
# ==============================================================================
# MART 5: SESSION_RECOMMENDATIONS - Recomendações Personalizadas
# ==============================================================================

query_mart_recommendations = """
WITH session_profiles AS (
    SELECT 
        datetime_utc,
        
        -- Condições básicas
        surf_quality_index,
        session_potential_score,
        wave_height_avg_m,
        wind_speed_kph,
        temperature_2m_celsius,
        precipitation_1h_mm,
        rating_score,
        
        -- Flags
        is_good_surf,
        is_surfable_size,
        is_good_wind,
        is_no_rain,
        
        -- Perfis de surfista
        CASE 
            WHEN wave_height_avg_m >= 2.0 AND rating_score >= 4 THEN 'Surfista Avançado'
            WHEN wave_height_avg_m >= 1.5 OR rating_score >= 3 THEN 'Surfista Intermediário'
            WHEN wave_height_avg_m >= 0.8 THEN 'Surfista Iniciante'
            ELSE 'Aula de Surf'
        END as target_surfer,
        
        -- Tipo de sessão recomendada
        CASE 
            WHEN surf_quality_index >= 85 AND wave_height_avg_m >= 1.5 THEN 'Sessão Épica'
            WHEN surf_quality_index >= 75 THEN 'Sessão de Performance'
            WHEN surf_quality_index >= 65 THEN 'Sessão Fun'
            WHEN surf_quality_index >= 50 THEN 'Sessão de Treino'
            ELSE 'Melhor Não Surfar'
        END as session_type,
        
        -- Recomendações específicas
        CASE 
            WHEN wave_height_avg_m >= 2.5 THEN 'Traga prancha de ondas grandes'
            WHEN wave_height_avg_m >= 1.5 THEN 'Prancha de performance'
            WHEN wave_height_avg_m >= 1.0 THEN 'Prancha funboard'
            ELSE 'Longboard ou foam'
        END as board_recommendation,
        
        -- Alertas especiais
        CASE 
            WHEN wind_speed_kph >= 25 THEN 'ALERTA: Vento forte'
            WHEN precipitation_1h_mm >= 5 THEN 'CHUVA: Considere se vale a pena'
            WHEN temperature_2m_celsius <= 15 THEN 'FRIO: Leve neoprene'
            WHEN temperature_2m_celsius >= 30 THEN 'CALOR: Hidrate-se bem'
            ELSE 'Condições normais'
        END as special_alert,
        
        -- Score personalizado para diferentes tipos
        -- Iniciante: prioriza segurança e ondas pequenas
        LEAST(100, 
            CASE WHEN wave_height_avg_m BETWEEN 0.5 AND 1.2 THEN 40 ELSE 10 END +
            CASE WHEN wind_speed_kph <= 15 THEN 25 ELSE 5 END +
            CASE WHEN is_no_rain THEN 20 ELSE 10 END +
            CASE WHEN temperature_2m_celsius BETWEEN 20 AND 28 THEN 15 ELSE 5 END
        ) as beginner_score,
        
        -- Avançado: prioriza qualidade e tamanho
        LEAST(100,
            surf_quality_index * 0.4 +
            CASE WHEN wave_height_avg_m >= 1.5 THEN 30 ELSE wave_height_avg_m * 15 END +
            CASE WHEN rating_score >= 4 THEN 20 ELSE rating_score * 4 END +
            CASE WHEN wind_speed_kph <= 20 THEN 10 ELSE 0 END
        ) as advanced_score
        
    FROM fact_surf_conditions
    WHERE surf_quality_index IS NOT NULL
),
recommendations AS (
    SELECT 
        datetime_utc,
        target_surfer,
        session_type,
        board_recommendation,
        special_alert,
        ROUND(surf_quality_index, 1) as surf_quality,
        ROUND(wave_height_avg_m, 2) as wave_height,
        ROUND(wind_speed_kph, 1) as wind_speed,
        ROUND(temperature_2m_celsius, 1) as temperature,
        ROUND(beginner_score, 1) as beginner_score,
        ROUND(advanced_score, 1) as advanced_score,
        
        -- Recomendação final
        CASE 
            WHEN advanced_score >= 80 THEN '🏆 GO SURF NOW!'
            WHEN advanced_score >= 65 THEN '🌟 Boa Sessão'
            WHEN beginner_score >= 70 THEN '🏄‍♂️ Perfeito para Iniciantes'
            WHEN beginner_score >= 50 THEN '📚 Bom para Aprender'
            ELSE '😴 Melhor Ficar em Casa'
        END as final_recommendation,
        
        -- Urgência da recomendação
        CASE 
            WHEN advanced_score >= 85 OR beginner_score >= 80 THEN 'Alta Prioridade'
            WHEN advanced_score >= 70 OR beginner_score >= 65 THEN 'Média Prioridade'
            ELSE 'Baixa Prioridade'
        END as priority_level
        
    FROM session_profiles
)
SELECT *
FROM recommendations
ORDER BY datetime_utc DESC
"""

# Executa e cria o mart
mart_session_recommendations = con.execute(query_mart_recommendations).fetch_df()

print("🎯 MART_SESSION_RECOMMENDATIONS CRIADO COM SUCESSO!")
print(f"Registros: {len(mart_session_recommendations):,}")

print("\n🏆 ÚLTIMAS SESSÕES ÉPICAS (GO SURF NOW!):")
epic_sessions = mart_session_recommendations[mart_session_recommendations['final_recommendation'] == '🏆 GO SURF NOW!']
if len(epic_sessions) > 0:
    display(epic_sessions[['datetime_utc', 'session_type', 'wave_height', 'wind_speed', 'board_recommendation', 'advanced_score']].head(10))
else:
    print("Nenhuma sessão épica recente - vamos ver as melhores disponíveis:")
    best_sessions = mart_session_recommendations.nlargest(5, 'advanced_score')
    display(best_sessions[['datetime_utc', 'final_recommendation', 'wave_height', 'wind_speed', 'advanced_score']])

print("\n📊 DISTRIBUIÇÃO DE RECOMENDAÇÕES:")
recommendation_dist = mart_session_recommendations['final_recommendation'].value_counts()
print(recommendation_dist)

print("\n🏄‍♂️ ANÁLISE POR PERFIL DE SURFISTA:")
surfer_analysis = mart_session_recommendations.groupby('target_surfer').agg({
    'surf_quality': 'mean',
    'beginner_score': 'mean',
    'advanced_score': 'mean',
    'wave_height': 'mean'
}).round(1)
display(surfer_analysis)

print("\n⚠️ ALERTAS ESPECIAIS MAIS COMUNS:")
alert_dist = mart_session_recommendations['special_alert'].value_counts()
print(alert_dist)

🎯 MART_SESSION_RECOMMENDATIONS CRIADO COM SUCESSO!
Registros: 590,819

🏆 ÚLTIMAS SESSÕES ÉPICAS (GO SURF NOW!):


,datetime_utc,session_type,wave_height,wind_speed,board_recommendation,advanced_score
2542,2022-12-26 16:00:00-03:00,Sessão de Performance,1.2,9.8,Prancha funboard,80.0
2543,2022-12-26 16:00:00-03:00,Sessão de Performance,1.2,9.8,Prancha funboard,80.0
2544,2022-12-26 16:00:00-03:00,Sessão de Performance,1.2,9.8,Prancha funboard,80.0
2545,2022-12-26 16:00:00-03:00,Sessão de Performance,1.2,9.8,Prancha funboard,80.0
2546,2022-12-26 16:00:00-03:00,Sessão de Performance,1.2,9.8,Prancha funboard,80.0
2547,2022-12-26 16:00:00-03:00,Sessão de Performance,1.2,9.8,Prancha funboard,80.0
2548,2022-12-26 16:00:00-03:00,Sessão de Performance,1.2,9.8,Prancha funboard,80.0
2549,2022-12-26 15:00:00-03:00,Sessão de Performance,1.2,9.8,Prancha funboard,80.0
2550,2022-12-26 15:00:00-03:00,Sessão de Performance,1.2,9.8,Prancha funboard,80.0
2551,2022-12-26 15:00:00-03:00,Sessão de Performance,1.2,9.8,Prancha funboard,80.0



📊 DISTRIBUIÇÃO DE RECOMENDAÇÕES:
final_recommendation
🏄‍♂️ Perfeito para Iniciantes    281103
🌟 Boa Sessão                     121471
📚 Bom para Aprender               86056
🏆 GO SURF NOW!                    51896
😴 Melhor Ficar em Casa            50293
Name: count, dtype: int64

🏄‍♂️ ANÁLISE POR PERFIL DE SURFISTA:


,surf_quality,beginner_score,advanced_score,wave_height
target_surfer,,,,
Aula de Surf,59.9,65.2,40.6,0.6
Surfista Avançado,100.0,60.0,100.0,2.4
Surfista Iniciante,71.1,74.9,51.4,1.1
Surfista Intermediário,90.5,58.3,73.3,1.5



⚠️ ALERTAS ESPECIAIS MAIS COMUNS:
special_alert
ALERTA: Vento forte                319214
Condições normais                  265261
FRIO: Leve neoprene                  3766
CHUVA: Considere se vale a pena      2576
CALOR: Hidrate-se bem                   2
Name: count, dtype: int64


In [27]:
# ==============================================================================
# MART 6: WEATHER_INSIGHTS - Análise Avançada de Padrões Climáticos  
# (Incluindo análise de chuva + vento como você sugeriu!)
# ==============================================================================

query_mart_weather_insights = """
WITH weather_combos AS (
    SELECT 
        f.*,
        dw.wind_category,
        dw.wind_direction_category,
        dw.temperature_category,
        dw.precipitation_category,
        dw.weather_rating,
        dw.avg_weather_score,
        
        -- Combinações especiais baseadas na sua observação
        CASE 
            WHEN f.precipitation_1h_mm > 0 AND f.wind_speed_kph <= 15 THEN 'Chuva + Vento Calmo'
            WHEN f.precipitation_1h_mm > 0 AND f.wind_speed_kph > 15 THEN 'Chuva + Vento Forte'
            WHEN f.precipitation_1h_mm = 0 AND f.wind_speed_kph <= 15 THEN 'Sem Chuva + Vento Calmo'
            WHEN f.precipitation_1h_mm = 0 AND f.wind_speed_kph > 15 THEN 'Sem Chuva + Vento Forte'
            ELSE 'Indefinido'
        END as rain_wind_combo,
        
        -- Offshore/Onshore analysis
        CASE 
            WHEN dw.wind_direction_category = 'Oeste' THEN 'Offshore (Ideal)'
            WHEN dw.wind_direction_category = 'Leste' THEN 'Onshore (Ruim)'
            WHEN dw.wind_direction_category = 'Sul' THEN 'Side-shore Sul'
            ELSE 'Side-shore Norte'
        END as wind_effect,
        
        -- Temperatura ideal para surf
        CASE 
            WHEN f.temperature_2m_celsius BETWEEN 22 AND 26 THEN 'Temperatura Ideal'
            WHEN f.temperature_2m_celsius BETWEEN 18 AND 30 THEN 'Temperatura Boa'
            WHEN f.temperature_2m_celsius < 18 THEN 'Frio'
            ELSE 'Muito Quente'
        END as temp_comfort,
        
        -- Pressão atmosférica e swells
        CASE 
            WHEN f.pressure_msl_hpa < 1010 THEN 'Baixa Pressão (Tempestade)'
            WHEN f.pressure_msl_hpa > 1020 THEN 'Alta Pressão (Estável)'
            ELSE 'Pressão Normal'
        END as pressure_condition
        
    FROM fact_surf_conditions f
    LEFT JOIN dim_weather dw ON f.weather_key = dw.weather_key
    WHERE f.weather_key IS NOT NULL
),
weather_stats AS (
    SELECT 
        rain_wind_combo,
        wind_effect,
        temp_comfort,
        pressure_condition,
        weather_rating,
        
        -- Estatísticas básicas
        COUNT(*) as total_sessions,
        AVG(surf_quality_index) as avg_surf_quality,
        AVG(session_potential_score) as avg_session_potential,
        AVG(wave_height_avg_m) as avg_wave_height,
        AVG(wind_speed_kph) as avg_wind_speed,
        AVG(temperature_2m_celsius) as avg_temperature,
        AVG(precipitation_1h_mm) as avg_precipitation,
        
        -- Taxa de sucesso
        AVG(CASE WHEN is_good_surf THEN 1.0 ELSE 0.0 END) * 100 as success_rate,
        COUNT(CASE WHEN is_good_surf THEN 1 END) as good_sessions,
        
        -- Análise de extremos
        MAX(surf_quality_index) as best_session,
        MIN(surf_quality_index) as worst_session,
        
        -- Consistência
        STDDEV(surf_quality_index) as quality_variance
        
    FROM weather_combos
    GROUP BY rain_wind_combo, wind_effect, temp_comfort, pressure_condition, weather_rating
)
SELECT 
    rain_wind_combo,
    wind_effect,
    temp_comfort,
    weather_rating,
    total_sessions,
    ROUND(avg_surf_quality, 1) as avg_surf_quality,
    ROUND(avg_session_potential, 1) as avg_session_potential,
    ROUND(success_rate, 2) as success_rate_pct,
    good_sessions,
    ROUND(avg_wind_speed, 1) as avg_wind_speed,
    ROUND(avg_temperature, 1) as avg_temperature,
    ROUND(avg_precipitation, 2) as avg_precipitation,
    ROUND(best_session, 1) as best_session_quality,
    
    -- Ranking por qualidade
    RANK() OVER (ORDER BY avg_surf_quality DESC) as quality_rank,
    
    -- Insight da combinação
    CASE 
        WHEN rain_wind_combo = 'Chuva + Vento Calmo' AND avg_surf_quality >= 70 THEN '🌧️✨ Teoria Confirmada!'
        WHEN wind_effect = 'Offshore (Ideal)' AND avg_surf_quality >= 75 THEN '💨🏄‍♂️ Offshore Magic'
        WHEN temp_comfort = 'Temperatura Ideal' AND avg_surf_quality >= 70 THEN '🌡️👌 Conforto Total'
        WHEN success_rate >= 3 THEN '🎯 Combo Confiável'
        ELSE '📊 Combo Regular'
    END as weather_insight,
    
    -- Recomendação
    CASE 
        WHEN avg_surf_quality >= 75 AND success_rate >= 2 THEN '🏆 Procure Esta Condição!'
        WHEN avg_surf_quality >= 65 THEN '⚡ Condição Boa'
        WHEN avg_surf_quality >= 55 THEN '👌 Condição OK'
        ELSE '🚫 Evite'
    END as recommendation

FROM weather_stats
WHERE total_sessions >= 5  -- Filtra combinações com dados suficientes
ORDER BY avg_surf_quality DESC
"""

# Executa e cria o mart
mart_weather_insights = con.execute(query_mart_weather_insights).fetch_df()

print("🌤️ MART_WEATHER_INSIGHTS CRIADO COM SUCESSO!")
print(f"Registros: {len(mart_weather_insights)}")

print("\n🏆 TOP 10 MELHORES COMBINAÇÕES CLIMÁTICAS:")
display(mart_weather_insights[['rain_wind_combo', 'wind_effect', 'avg_surf_quality', 'success_rate_pct', 'weather_insight', 'recommendation']].head(10))

print("\n🌧️ TESTANDO A TEORIA: CHUVA + VENTO CALMO")
print("=" * 60)
rain_calm = mart_weather_insights[mart_weather_insights['rain_wind_combo'] == 'Chuva + Vento Calmo']
no_rain_calm = mart_weather_insights[mart_weather_insights['rain_wind_combo'] == 'Sem Chuva + Vento Calmo']

if len(rain_calm) > 0 and len(no_rain_calm) > 0:
    print(f"🌧️😌 Chuva + Vento Calmo: {rain_calm.iloc[0]['avg_surf_quality']:.1f} qualidade média")
    print(f"☀️😌 Sem Chuva + Vento Calmo: {no_rain_calm.iloc[0]['avg_surf_quality']:.1f} qualidade média")
    print(f"💨 Vento médio com chuva: {rain_calm.iloc[0]['avg_wind_speed']:.1f} km/h")
    print(f"💨 Vento médio sem chuva: {no_rain_calm.iloc[0]['avg_wind_speed']:.1f} km/h")
    
    if rain_calm.iloc[0]['avg_surf_quality'] >= no_rain_calm.iloc[0]['avg_surf_quality'] * 0.95:
        print("\n✅ TEORIA CONFIRMADA: Chuva com vento calmo pode ser TÃO BOA quanto sem chuva!")
    elif rain_calm.iloc[0]['avg_wind_speed'] < no_rain_calm.iloc[0]['avg_wind_speed']:
        print("\n🌧️✨ PARCIALMENTE CONFIRMADA: Chuva = menos vento, mas pode impactar outros fatores")

print("\n💨 ANÁLISE DE DIREÇÃO DO VENTO:")
wind_analysis = mart_weather_insights.groupby('wind_effect').agg({
    'avg_surf_quality': 'mean',
    'success_rate_pct': 'mean',
    'total_sessions': 'sum'
}).round(1).sort_values('avg_surf_quality', ascending=False)
display(wind_analysis)

print("\n🌡️ ANÁLISE DE CONFORTO TÉRMICO:")
temp_analysis = mart_weather_insights.groupby('temp_comfort').agg({
    'avg_surf_quality': 'mean',
    'success_rate_pct': 'mean',
    'total_sessions': 'sum'
}).round(1).sort_values('avg_surf_quality', ascending=False)
display(temp_analysis)

🌤️ MART_WEATHER_INSIGHTS CRIADO COM SUCESSO!
Registros: 366

🏆 TOP 10 MELHORES COMBINAÇÕES CLIMÁTICAS:


,rain_wind_combo,wind_effect,avg_surf_quality,success_rate_pct,weather_insight,recommendation
0,Sem Chuva + Vento Forte,Offshore (Ideal),100.0,0.00,💨🏄‍♂️ Offshore Magic,⚡ Condição Boa
1,Sem Chuva + Vento Calmo,Side-shore Norte,100.0,33.33,🌡️👌 Conforto Total,🏆 Procure Esta Condição!
2,Sem Chuva + Vento Calmo,Side-shore Norte,100.0,33.33,🌡️👌 Conforto Total,🏆 Procure Esta Condição!
3,Chuva + Vento Calmo,Offshore (Ideal),100.0,8.00,🌧️✨ Teoria Confirmada!,🏆 Procure Esta Condição!
4,Chuva + Vento Calmo,Side-shore Norte,100.0,0.00,🌧️✨ Teoria Confirmada!,⚡ Condição Boa
5,Chuva + Vento Calmo,Side-shore Norte,100.0,0.00,🌧️✨ Teoria Confirmada!,⚡ Condição Boa
6,Chuva + Vento Calmo,Side-shore Norte,100.0,0.00,🌧️✨ Teoria Confirmada!,⚡ Condição Boa
7,Sem Chuva + Vento Calmo,Offshore (Ideal),100.0,0.00,💨🏄‍♂️ Offshore Magic,⚡ Condição Boa
8,Chuva + Vento Calmo,Offshore (Ideal),100.0,0.00,🌧️✨ Teoria Confirmada!,⚡ Condição Boa
9,Chuva + Vento Calmo,Side-shore Norte,100.0,0.00,🌧️✨ Teoria Confirmada!,⚡ Condição Boa



🌧️ TESTANDO A TEORIA: CHUVA + VENTO CALMO
🌧️😌 Chuva + Vento Calmo: 100.0 qualidade média
☀️😌 Sem Chuva + Vento Calmo: 100.0 qualidade média
💨 Vento médio com chuva: 13.4 km/h
💨 Vento médio sem chuva: 11.7 km/h

✅ TEORIA CONFIRMADA: Chuva com vento calmo pode ser TÃO BOA quanto sem chuva!

💨 ANÁLISE DE DIREÇÃO DO VENTO:


,avg_surf_quality,success_rate_pct,total_sessions
wind_effect,,,
Offshore (Ideal),83.8,3.9,22860
Side-shore Norte,81.7,6.1,260976
Onshore (Ruim),80.3,0.0,116891
Side-shore Sul,76.3,0.1,190079



🌡️ ANÁLISE DE CONFORTO TÉRMICO:


,avg_surf_quality,success_rate_pct,total_sessions
temp_comfort,,,
Frio,85.2,1.9,127044
Temperatura Boa,80.6,2.5,257634
Temperatura Ideal,76.4,3.2,206109
Muito Quente,64.6,0.0,19


In [32]:
# ==============================================================================
# 💾 SALVANDO TODOS OS DATA MARTS COMO DELTA TABLES
# ==============================================================================

print("💾 SALVANDO DATA MARTS COMO DELTA TABLES...")
print("=" * 60)

# Lista de todos os marts criados
marts_to_save = {
    'mart_best_surf_times': mart_best_surf_times,
    'mart_rain_analysis': mart_rain_analysis, 
    'mart_swell_patterns': mart_swell_patterns,
    'mart_seasonal_insights': mart_seasonal_insights,
    'mart_session_recommendations': mart_session_recommendations,
    'mart_weather_insights': mart_weather_insights
}

# Salva cada mart como Delta Table
for mart_name, mart_df in marts_to_save.items():
    mart_path = os.path.join(GOLD_DIR, mart_name)
    
    try:
        write_deltalake(mart_path, mart_df, mode='overwrite')
        print(f"✅ {mart_name}: {len(mart_df):,} registros salvos em '{mart_path}'")
    except Exception as e:
        print(f"❌ Erro ao salvar {mart_name}: {e}")

print("\n🏆 RESUMO DOS DATA MARTS CRIADOS:")
print("=" * 60)
for mart_name, mart_df in marts_to_save.items():
    print(f"📊 {mart_name.upper()}")
    print(f"   📈 Registros: {len(mart_df):,}")
    print(f"   💾 Colunas: {len(mart_df.columns)}")
    if 'recommendation' in mart_df.columns:
        top_recommendation = mart_df['recommendation'].iloc[0] if len(mart_df) > 0 else 'N/A'
        print(f"   🎯 Top Insight: {top_recommendation}")
    print()

print("🎉 TODOS OS DATA MARTS SALVOS COM SUCESSO!")
print("🚀 Prontos para análises avançadas, dashboards e ML!")

💾 SALVANDO DATA MARTS COMO DELTA TABLES...
✅ mart_best_surf_times: 24 registros salvos em 'data/gold/mart_best_surf_times'
✅ mart_rain_analysis: 5 registros salvos em 'data/gold/mart_rain_analysis'
✅ mart_swell_patterns: 84 registros salvos em 'data/gold/mart_swell_patterns'
✅ mart_seasonal_insights: 12 registros salvos em 'data/gold/mart_seasonal_insights'
✅ mart_session_recommendations: 590,819 registros salvos em 'data/gold/mart_session_recommendations'
✅ mart_weather_insights: 366 registros salvos em 'data/gold/mart_weather_insights'

🏆 RESUMO DOS DATA MARTS CRIADOS:
📊 MART_BEST_SURF_TIMES
   📈 Registros: 24
   💾 Colunas: 14
   🎯 Top Insight: 🏄‍♂️ Bom para Surf

📊 MART_RAIN_ANALYSIS
   📈 Registros: 5
   💾 Colunas: 12

📊 MART_SWELL_PATTERNS
   📈 Registros: 84
   💾 Colunas: 13
   🎯 Top Insight: 🏆 HUNT THIS SWELL!

📊 MART_SEASONAL_INSIGHTS
   📈 Registros: 12
   💾 Colunas: 21

📊 MART_SESSION_RECOMMENDATIONS
   📈 Registros: 590,819
   💾 Colunas: 13

📊 MART_WEATHER_INSIGHTS
   📈 Registro

# 🌟 CAMADA GOLD - Star Schema para Análise de Surf
## Criando modelo dimensional otimizado para BI e análises de negócio